In [1]:
# -*- coding: utf-8 -*-

'''
@Author   :   Corley Tang
@contact  :   cutercorleytd@gmail.com
@Github   :   https://github.com/corleytd
@Time     :   2023-04-19 10:21
@Project  :   Hands-on Deep Learning with PyTorch-classical_convolutional_neural_networks_and_model_architecture_evaluation
经典卷积神经网络与模型评估
'''

# 导入所需的库
from typing import Type, Union, List

import torch
from torch import nn
from torch.nn import functional as F
from torchinfo import summary

from utils.receptive_field import receptive_field

# 设置随机种子
torch.manual_seed(20230430)

## 1.复现传统经典架构
复现架构时的一些注意点如下：
1. 代码可以不自己写，但一定要了解此架构的核心思想，以及在此思想下其结构的排布。对于比较简单的网络（比如AlexNet），可以记住具体的层的结构，但对于复杂网络，应该以理解思想为主，看见代码能认出该网络，并且理解每层具体在做什么，可以与该网络的核心思想一致，同时对模型的各个模块进行分组，确定每个组应该实现的功能，以便更好地理解默写架构。
2. 了解此架构被提出的背景，即可了解它的突破与局限。
3. 发现架构与自己熟悉的不一致，先多了解不同的架构图，因为也有很多基于经典架构修改的变体同时可以参考提出框架的论文。
### 现代CNN的奠基者：LeNet5
使用卷积层和池化层搭建一个简单的卷积神经网络，架构如下：
![lenet5_structure](../assets/lenet5_structure.png)

可以看到，图像从左侧输入，从右侧输出，整个网络由2个卷积层、2个平均池化层和2个全连接层组成，每个卷积层和全连接层后都使用激活函数Tanh或Sigmoid。这个架构就是LeNet5架构，它在1998年被LeCun等人在论文《Gradient-Based Learning Applied to Document Recognition》中正式提出，被认为是现代卷积神经网络的奠基者，在LeNet5被提出后，几乎所有的卷积网络都会连用卷积层、池化层与全连接层（线性层）的范式。现在，这已经成为一种非常经典的架构：卷积层作为输入层，紧跟激活函数，池化层紧跟在一个或数个卷积+激活的结构之后；在卷积池化交替进行数次之后，转向线性层+激活函数，并使用线性层结尾，输出预测结果。下面进行复现。

In [2]:
class LeNet5(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.tanh1 = nn.Tanh()
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.tanh2 = nn.Tanh()
        self.pool2 = nn.AvgPool2d(2)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.tanh3 = nn.Tanh()
        self.fc2 = nn.Linear(120, 84)

    def forward(self, x):
        out = self.tanh1(self.conv1(x))
        out = self.pool1(out)
        out = self.tanh2(self.conv2(out))
        out = self.pool2(out)
        out = out.reshape(out.shape[0], -1)  # 将特征拉平
        out = self.tanh3(self.fc1(out))
        out = F.softmax(self.fc2(out), -1)
        return out

In [3]:
# 构建数据
images = torch.randint(0, 256, (16, 1, 32, 32), dtype=torch.float)
# 实例化模型
model = LeNet5()
# 前向传播
preds = model(images)
preds.shape  # (16, 84)

torch.Size([16, 84])

为了直观地查看模型结构和各部分的输出等信息，可以使用一些工具，如torchinfo库，通过`conda install torchinfo`命令进行安装即可。下面使用torchinfo库输出模型结构。

In [4]:
summary(model, input_size=(16, 1, 32, 32))

Layer (type:depth-idx)                   Output Shape              Param #
LeNet5                                   [16, 84]                  --
├─Conv2d: 1-1                            [16, 6, 28, 28]           156
├─Tanh: 1-2                              [16, 6, 28, 28]           --
├─AvgPool2d: 1-3                         [16, 6, 14, 14]           --
├─Conv2d: 1-4                            [16, 16, 10, 10]          2,416
├─Tanh: 1-5                              [16, 16, 10, 10]          --
├─AvgPool2d: 1-6                         [16, 16, 5, 5]            --
├─Linear: 1-7                            [16, 120]                 48,120
├─Tanh: 1-8                              [16, 120]                 --
├─Linear: 1-9                            [16, 84]                  10,164
Total params: 60,856
Trainable params: 60,856
Non-trainable params: 0
Total mult-adds (M): 6.76
Input size (MB): 0.07
Forward/backward pass size (MB): 0.83
Params size (MB): 0.24
Estimated Total Size (MB): 1.14

可以看到，输出信息中包含了每一层的类型、输出尺寸和参数量，同时还汇总了参数量等信息。

对于层数较少、层次简单的神经网络，可以通过逐层定义各层的方式实现，这样也能直观地看到卷积层、池化层、全连接层是如何连接在一起共同作用的，但是对于复杂的模型可能不太适用。如今，LeNet5虽然是很简单并且有些弱小的卷积网络，当加入学习率衰减等优化方法并训练恰当时，单一的LeNet5模型在Fashion-MNIST数据集上也可以获得超过91%的训练准确率，可参考[https://www.kaggle.com/code/jaeboklee/pytorch-fashion-mnist-lenet5-ensemble/](https://www.kaggle.com/code/jaeboklee/pytorch-fashion-mnist-lenet5-ensemble/)，与线性层相比，提升了约5%，LeNet5虽然简单，但也展示了卷积神经网络的实力和潜力。
### 从浅层到深度：AlexNet
在21世纪的前十年，大多数人并不相信机器提取的特征能够比人亲自提取的特征更强大，LeNet5也确实无法在复杂任务上战胜传统计算机视觉方法，直到2012年AlexNet横空出世，第一次证明了，当数据量达标、训练得当时，卷积神经网络提取的特征效果远远胜过人类提取的特征。AlexNet诞生于有“视觉界奥林匹克”之称的大规模视觉识别挑战比赛ILSVRC（ImageNet Large Scale Visual Recognition Challenge），ILSVRC使用ImageNet数据集，共有1400多万幅图像，共2万多个类别，图像的尺寸有224×224、227×227、256×256、299×299等不同类型，参赛队伍使用大约包含一百万张图片的训练集来训练模型，并识别出测试集中的1000个类别，再按错误率从低到高进行排名。在AlexNet出现之前，最好成绩一直由手工提取特征+支持向量机的算法获得，最低错误率为25.8%，2012年，AlexNet进入ILSVRC竞赛一下将错误率降低到了15.3%，深度学习首次入场就取得了10个百分点的提升，这给整个图像领域带来了巨大的震撼，从此图像领域的前沿研究就全面向深度学习倾斜。AlexNet架构如下：
![alexnet_structure](../assets/alexnet_structure.png)

可以看到，AlexNet总共有11层，其中有5个卷积层、3个池化层、2个全连接隐藏层，1个全连接输出层，在全连接层的前后还使用了Dropout层。在认识架构时，不用去纠结整体的层数，可以从层与层之间的组合入手：LeNet5的架构可以打包成3个组合，即**输入→(卷积+池化)→(卷积+池化)→(线性x2)→输出**；AlexNet的架构可以打包成4个组合，即**输入→(卷积+池化)→(卷积+池化)→(卷积x3+池化)→(线性x3)→输出**。

AlexNet几乎从算法、算力、数据、架构技巧等各个方面做出了探索，从而影响了现代卷积神经网络的发展。和6层的LeNet5比起来，AlexNet主要提出了如下创新：
1. 卷积核更小、网络更深、通道数更多：基于ImageNet数据集训练的AlexNet最大的卷积核只有11x11，且在第二个卷积层就改用5x5，剩下的层中都使用3x3的卷积核，图像尺寸/核尺寸至少也超过20：1。小卷积核让网络更深，但也让特征图的尺寸变得很小，为了让信息尽可能地被捕获，AlexNet也使用了更多的通道来弥补特征图变小带来的不足。小卷积核、多通道、更深的网络，这些都成为了卷积神经网络后续发展的指导方向。
2. 使用了ReLU激活函数，摆脱Sigmoid与Tanh的各种问题。
3. 使用了Dropout层来控制模型复杂度，控制过拟合。
4. 引入了大量传统或新兴的图像增强技术来扩大数据集，进一步缓解过拟合。
5. 使用了Overlap Pooling：一般的池化层在进行扫描时，都是步幅 >= 核尺寸，而在AlexNet中，池化层中的步幅小于核尺寸，使得池化过程中的扫描区域出现重叠（overlap），以此来缓解过拟合。
6. 提出使用GPU对网络进行训练，使得Proper Training适当的训练成为可能。

下面复现AlexNet模型。

In [5]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=1000, dropout=0.5):
        super().__init__()
        self.dropout = dropout

        #为了处理尺寸较大的图片，先使用较大的卷积核和较大的步长来快速降低特征图的尺寸，同时使用比较多的通道数来弥补降低尺寸造成的信息损失
        self.conv1 = nn.Conv2d(3, 96, 11, stride=4)
        self.pool1 = nn.MaxPool2d(3, stride=2)

        # 将特征图尺寸缩小到27x27，计算量可控，使得可以进行特征提取，同时卷积核、步长恢复到常用尺寸，进一步扩大通道来提取图像信息
        self.conv2 = nn.Conv2d(96, 256, 5, padding=2)
        self.pool2 = nn.MaxPool2d(3, stride=2)

        # 连续用多个卷积层尽可能多地提取特征，这里步长为1，所以设置的Kernel为5、Padding为2和Kernel为3、Padding为1都可以维持特征图的大小不变
        self.conv3 = nn.Conv2d(256, 384, 3, padding=1)
        self.conv4 = nn.Conv2d(384, 384, 3, padding=1)
        self.conv5 = nn.Conv2d(384, 256, 3, padding=1)
        self.pool3 = nn.MaxPool2d(3, stride=2)

        # 进入全连接层，进行信息汇总
        self.fc1 = nn.Linear(256 * 6 * 6, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, num_classes)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = self.pool1(out)

        out = F.relu(self.conv2(out))
        out = self.pool2(out)

        out = F.relu(self.conv3(out))
        out = F.relu(self.conv4(out))
        out = F.relu(self.conv5(out))
        out = self.pool3(out)

        out = out.view(out.shape[0], -1)  # 将数据的特征拉平
        out = F.dropout(out, p=self.dropout)
        out = F.relu(F.dropout(self.fc1(out), p=self.dropout))
        out = F.relu(self.fc2(out))
        out = F.softmax(self.fc3(out), -1)
        return out

In [6]:
# 构建数据
images = torch.randint(0, 256, (16, 3, 227, 227), dtype=torch.float)
# 实例化模型
model = AlexNet(dropout=0.2)
# 前向传播
preds = model(images)
preds.shape  # (16, 1000)

torch.Size([16, 1000])

In [7]:
# 使用torchinfo查看模型结构
summary(model, (16, 3, 227, 227))

Layer (type:depth-idx)                   Output Shape              Param #
AlexNet                                  [16, 1000]                --
├─Conv2d: 1-1                            [16, 96, 55, 55]          34,944
├─MaxPool2d: 1-2                         [16, 96, 27, 27]          --
├─Conv2d: 1-3                            [16, 256, 27, 27]         614,656
├─MaxPool2d: 1-4                         [16, 256, 13, 13]         --
├─Conv2d: 1-5                            [16, 384, 13, 13]         885,120
├─Conv2d: 1-6                            [16, 384, 13, 13]         1,327,488
├─Conv2d: 1-7                            [16, 256, 13, 13]         884,992
├─MaxPool2d: 1-8                         [16, 256, 6, 6]           --
├─Linear: 1-9                            [16, 4096]                37,752,832
├─Linear: 1-10                           [16, 4096]                16,781,312
├─Linear: 1-11                           [16, 1000]                4,097,000
Total params: 62,378,344
Trainable p

可以看到，与LeNet5相比，AlexNet虽然只增加了5层，但是参数量却增加了1000倍，所以计算量也会大幅增加，对算力提出了更高的要求。今如今，只要有足够的算力和数据，即便是几亿参数的巨量网络也是可以训练的，AlexNet所带来的观念上的转变成为了真正推动卷积神经网络向前发展的核心。

## 2.模型架构对学习能力/鲁棒性的影响
对于经典机器学习算法而言，算法的学习能力大多由其根本的数学逻辑决定，不同算法之间的数学逻辑不可轻易切换或混用，无论我们如何调整参数，也不会轻易改变机器学习算法的基本逻辑。而在深度学习中，每个架构都是为当前的数据“量身定制”的，虽然存在一些底层的逻辑和思路，但一般来说，都没有严格的数学逻辑，也没有必须遵循的信息处理流程，虽然也会依赖经典架构来进行建模，但当输入数据发生变化之后，任何经典架构都可能需要调整、甚至变得完全不适用，因此改变经典架构的能力很重要，根据输入数据的不同，可能需要改变层的数目、层的参数、层的排列组合、串联并联方式等内容。同时，深度学习模型需要大量数据进行训练，因此模型的效率也很关键，在达成相似效果的前提下，参数更少的CNN会更受欢迎。
### 深度
更深的网络会展现出更强大的学习能力，但是在CNN中输入图像的尺寸会限制可以选择的深度。在特征图进入最终的全连接层之前，其最小尺寸大约需要被控制在5x5-9x9之间，常见尺寸为5x5、7x7、9x9。一个不重叠的池化层可以将特征图的宽高折半，而步长为2的卷积层也可以轻松将特征图的宽高折半，池化层的各项相关参数很少小于(2, 2)，因此只能依赖卷积层来控制特征图尺寸的缩小速度。为了特征图尺寸不“折半”，步长只能是1；为了深度，卷积核最好保持小尺寸，而这又限制了“填充”无法被设定得太大。池化层的Padding参数值必须小于池化核尺寸的1/2，池化核才会捕捉到足够的信息，卷积层的Padding参数值虽然不受程序强制限制，但一般也会设置为至少是小于卷积核尺寸。

所以，在追求“深度”的过程中，卷积层的参数选择其实很少，常见的特征图尺寸变化如下：
1. 宽高分别折半、或缩小更多
2. 不使用池化层，利用填充与卷积核尺寸的搭配，令特征图每经过一次卷积层，就缩小2个或4个像素——递减架构
3. 利用填充与卷积核尺寸的搭配，令特征图每经过一次卷积层，尺寸不变，缩小特征图的工作由池化层来完成——重复架构

递减架构和重复架构可以尽最大程度地增加深度，但是也会有一些区别：
- 递减架构可以让网络非常“深”
- 重复架构的参数量和计算量更小

重复架构的计算效率更高，可以满足更多的应用场景，同时一般情况下重复架构的鲁棒性会更好，在测试集上的表现会更高，因此被选择得更多。也可以两者结合，即在重复架构上不断加深网络，VGGNet就是以卷积、池化层为基础继续奋斗在“加深深度”这一方向并取得成功的架构，其核心思想是使用多个连续且保持特征图尺寸不变的卷积层来增加深度，以增加算法的学习能力。VGG有6种架构，其中经典的有3种：VGG13包含10个卷积层+3个线性层；VGG16包含13个卷积层+3个线性层；VGG19包含16个卷积层+3个线性层。VGG13的效果远远不如19层和16层的网络，VGG19效果更好，但在实际应用中拔得头筹的是VGG16，因为效果提升对于增加的计算量来说是很微小的、收益很低。随着深度的加深，模型的学习能力大概率会增强，但深度与模型效果之间的关系不是线性的，可以增长的边际准确率是在递减的，准确率的变化会逐渐趋于平缓，但参数量却是高速增加的。在不改变原始卷积层输入输出机制的前提下，通过增加卷积层的数目来增加深度，会很快让模型效果和性能都达到上限，因此深度并不能高效提升模型的效果，需要先降低模型的训练成本，才能够追求更深的神经网络，如果想要通过“加深”卷积神经网络来实现网络效果的飞跃，应该让层数有一个大的变化。所以，为了计算效率，不必强行加深网络，若的确追求更高的准确率，则可以考虑换成更高级的架构或者对数据进行预处理。

之所以度网络对比浅层网络更具优越性，有以下几个方面的可能原因：
1. 在同样的资源支持下，深度网络解决复杂问题的能力高于浅层网络：深度网络展现出对复合函数的拟合能力，更深的网络能够拟合更复杂的复合函数，而浅层网络的这种能力却不明显。在复合函数中埋藏越深的关系，也可能需要越多的网络层数来进行拟合。
2. 深度网络可以快速降低优化算法进入一个很大的局部最小值的概率：神经网络的深度越深时，权重空间更加复杂，同时损失函数的众多局部最小值的大小将会变得比较接近（鞍点会变平、整个函数的图像会变得平滑），并且随着深度加深，局部最小值的数量会越来越少、数值也越来越接近，这就降低了优化算法走入一个值很大的局部最小值的可能性。
3. 更深的网络能带来更大的感受野，而更大的感受野能带来更好的模型效果。

#### 复现VGG16
VGG架构对于神经网络研究和使用都有重要的意义，它不仅简单、有效，而且非常适合用来做各种实验和测试。VGG16的架构简化为：输入→（卷积x2+池化）x2 →（卷积x3+池化）x3 → FC层x3 →输出，具体如下：
![vgg16_structure](../assets/vgg16_structure.png)

其中，激活函数都是ReLU函数，后3个全连接层中的前2个全连接层前有Dropout层（p=0.5）。复现如下。

In [8]:
class VGG16(nn.Module):
    def __init__(self, num_classes=1000, dropout=0.5):
        super().__init__()
        self.dropout = dropout

        # 块1
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.conv2 = nn.Conv2d(64, 64, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2)

        # 块2
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 128, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2)

        # 块3
        self.conv5 = nn.Conv2d(128, 256, 3, padding=1)
        self.conv6 = nn.Conv2d(256, 256, 3, padding=1)
        self.conv7 = nn.Conv2d(256, 256, 3, padding=1)
        self.pool3 = nn.MaxPool2d(2)

        # 块4
        self.conv8 = nn.Conv2d(256, 512, 3, padding=1)
        self.conv9 = nn.Conv2d(512, 512, 3, padding=1)
        self.conv10 = nn.Conv2d(512, 512, 3, padding=1)
        self.pool4 = nn.MaxPool2d(2)

        # 块5
        self.conv11 = nn.Conv2d(512, 512, 3, padding=1)
        self.conv12 = nn.Conv2d(512, 512, 3, padding=1)
        self.conv13 = nn.Conv2d(512, 512, 3, padding=1)
        self.pool5 = nn.MaxPool2d(2)

        # 全连接层
        self.fc1 = nn.Linear(512 * 7 * 7, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, num_classes)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.relu(self.conv2(out))
        out = self.pool1(out)

        out = F.relu(self.conv3(out))
        out = F.relu(self.conv4(out))
        out = self.pool2(out)

        out = F.relu(self.conv5(out))
        out = F.relu(self.conv6(out))
        out = F.relu(self.conv7(out))
        out = self.pool3(out)

        out = F.relu(self.conv8(out))
        out = F.relu(self.conv9(out))
        out = F.relu(self.conv10(out))
        out = self.pool4(out)

        out = F.relu(self.conv11(out))
        out = F.relu(self.conv12(out))
        out = F.relu(self.conv13(out))
        out = self.pool5(out)

        out = out.reshape(out.shape[0], -1)
        out = F.dropout(out, self.dropout)
        out = F.relu(self.fc1(out))
        out = F.dropout(out, self.dropout)
        out = F.relu(self.fc2(out))
        out = F.softmax(self.fc3(out), -1)
        return out

In [9]:
# 构建数据
images = torch.randint(0, 256, (16, 3, 224, 224), dtype=torch.float)
# 实例化模型
model = VGG16(dropout=0.2)
# 前向传播
preds = model(images)
preds.shape  # (16, 1000)

torch.Size([16, 1000])

In [10]:
# 使用torchinfo查看模型结构
summary(model, (16, 3, 224, 224), device='cpu')

Layer (type:depth-idx)                   Output Shape              Param #
VGG16                                    [16, 1000]                --
├─Conv2d: 1-1                            [16, 64, 224, 224]        1,792
├─Conv2d: 1-2                            [16, 64, 224, 224]        36,928
├─MaxPool2d: 1-3                         [16, 64, 112, 112]        --
├─Conv2d: 1-4                            [16, 128, 112, 112]       73,856
├─Conv2d: 1-5                            [16, 128, 112, 112]       147,584
├─MaxPool2d: 1-6                         [16, 128, 56, 56]         --
├─Conv2d: 1-7                            [16, 256, 56, 56]         295,168
├─Conv2d: 1-8                            [16, 256, 56, 56]         590,080
├─Conv2d: 1-9                            [16, 256, 56, 56]         590,080
├─MaxPool2d: 1-10                        [16, 256, 28, 28]         --
├─Conv2d: 1-11                           [16, 512, 28, 28]         1,180,160
├─Conv2d: 1-12                           [16, 5

可以看到，此时模型的参数量已经过亿，达到1.38亿，与AlexNet相比增加了1倍多，此时也需要更多的计算资源才能支持模型的训练。

### 感受野
#### 认识感受野
对感受野更加严谨的定义：在深度卷积神经网络中，每个神经元节点都对应着输入图像上的某个区域，且该神经元仅受这个区域中的图像内容的影响，这个区域iu称为神经元的感受野。由于卷积神经网络“稀疏交互”的特性，CNN中的神经元只受到原始图像上一部分数据的影响，而这部分数据其实就是神经元在生成过程中、使用卷积操作时扫描到的那部分原始数据，这部分数据所在的区域也就是感受野。一个4层卷积的递减架构感受野变化如下：
![decreasing_receptive_field](../assets/decreasing_receptive_field.png)

可以看到，随着深度的加深，神经元上的感受野会越来越大，其所携带的原始数据的信息会越来越多，为了让神经元在做出判断之前能够获取尽量多的信息，在被输入到FC层之前的感受野越大越好。一个表现优异的模型在FC层前的感受野一定是非常大的，巨大的感受野是模型表现好的必要条件，对于像素级别的预测任务（密集预测任务），如需要给原始图像中每个像素点进行分类的语义分割任务、语音处理领域的立体声和光流估计任务等，FC层之前的感受野的大小至关重要，对于其他任务，也会尽量保证分类前感受野的大小是足够大的。
#### 感受野的性质
- 深度越大，感受野越大，池化层放大感受野的效率更高
  重复架构（如VGG）的感受野变化如下：
  ![repetitive_receptive_field](../assets/repetitive_receptive_field.png)

  可以看到，递减架构在经历4个卷积层后，图像的尺寸由22x22下降到了14x14、感受野尺寸为9x9，而重复架构经过4层卷积层和1个池化层之后，图像的尺寸下降到了11x11，且第四个卷积层生成的特征图上的神经元的感受野达到了12x12，比递减架构相同卷积层下的感受野要大，因此重复架构的预测效果比递减架构更好，主要是因为池化层在将特征图尺寸减半的同时，却能够将感受野的宽和高都扩大一倍令重复架构放大感受野的效率更高，这让重复架构更有优势。
- 放大感受野，没有极限
  感受野的尺寸没有上限：对于边缘神经元来说，它的感受野却会超出原始图像一部分，超出的图像就相当于没有值、全为0，表现在图像上就是黑框。同时，随着网络深度的逐渐加深，感受野中超出原始图像的部分会越来越多、黑边会越来越宽。一些经典的卷积神经网络感受野尺寸如下：

  | ConvNet Model         | Receptive Field (r) | Effective Stride (S) | Effective Padding (P) | Model Year  |
  | :------: | :------: | :------: | :------: | :------: |
  | alexnet_v2           | 195                       | 32                         | 64                          | [2014](https://arxiv.org/abs/1404.5997v2) |
  | vgg_16               | 212                       | 32                         | 90                          | [2014](https://arxiv.org/abs/1409.1556)   |
  | mobilenet_v1         | 315                       | 32                         | 126                         | [2017](https://arxiv.org/abs/1704.04861)  |
  | mobilenet_v1_075    | 315                       | 32                         | 126                         | [2017](https://arxiv.org/abs/1704.04861)  |
  | resnet_v1_50        | 483                       | 32                         | 239                         | [2015](https://arxiv.org/abs/1512.03385)  |
  | inception_v2         | 699                       | 32                         | 318                         | [2015](https://arxiv.org/abs/1502.03167)  |
  | resnet_v1_101       | 1027                      | 32                         | 511                         | [2015](https://arxiv.org/abs/1512.03385)  |
  | inception_v3         | 1311                      | 32                         | 618                         | [2015](https://arxiv.org/abs/1512.00567)  |
  | resnet_v1_152       | 1507                      | 32                         | 751                         | [2015](https://arxiv.org/abs/1512.03385)  |
  | resnet_v1_200       | 1763                      | 32                         | 879                         | [2015](https://arxiv.org/abs/1512.03385)  |
  | inception_v4         | 2071                      | 32                         | 998                         | [2016](https://arxiv.org/abs/1602.07261)  |
  | inception_resnet_v2 | 3039                      | 32                         | 1482                        | [2016](https://arxiv.org/abs/1602.07261)  |

  更多关于卷积神经网络的感受野信息可参考[https://distill.pub/2019/computing-receptive-fields/](https://distill.pub/2019/computing-receptive-fields/)。
- 关注中心，周围模糊
  人眼的感受野也存在“中间清晰，周围模糊”，即关注视野中间的事物，而可能并不能看清周围的事物。卷积神经网络的感受野也是一样，对于特征图来说，每个神经元所捕捉到的感受野区域不同，但这些区域是会有重叠的，并且越是位于中间的像素，被扫描的次数越多，感受野重叠也会越多，对整个特征图来说，重叠越多的部分，信息就越饱满，“看得就越清晰”，而重叠较少的部分，信息就比较稀疏，因此就会“模糊”。因此，位于图像中间的像素有更多可以影响最终特征图的“路径”，它们对最终特征图的影响更大，对卷积网络的分类造成的影响也会更大。有效感受野（Effective Receptive Fields，ERF）是一个比感受野小很多的区域，在这个有效感受野中所有的像素都对分类有较强的影响。因为中间清晰、两边模糊的性质和有效感受野的存在，必须尽量让图像的有效信息集中在感受野的中心，这些有效信息越集中，感受野就越能“看清”这些信息，越能提取出有效的特征。因此最有效的做法就是使用远远超出图像尺寸的感受野，而将图像信息固定在感受野中心，让本来应该“模糊”的部分全都被黑边所替代。
#### 扩大感受野：膨胀卷积
常见的扩大感受野的方法包括：
- 加深卷积神经网络的深度：理论上来说，每增加一个卷积层，感受野的宽和高就会按照卷积核的尺寸-1线性增加
- 使用池化层或其他快速消减特征图尺寸的技术
- 使用更加丰富的卷积操作，如膨胀卷积dilated convolution、残差连接等

膨胀卷积又叫做空洞卷积，它通过在感受野上使需要计算点“膨胀”的方式来“扩大”卷积核可以扫描的区域。在感受野上，需要参与卷积计算的任意一个像素都是计算点，以计算点为中心向外扩充像素点的行为叫做膨胀：以计算点为中心，膨胀率（dilation rate）为1时，计算点自身就是全部面积；膨胀率为2时，在计算点周边扩充一圈像素；当膨胀率为3时，在计算点周边填充2圈像素，以此类推。膨胀卷积就是在每个参与卷积计算的计算点上做膨胀操作，让计算点与计算点之间出现空洞，并跳过空洞进行计算的卷积方式。图示如下：
![dilation_convolution_demo](../assets/dilation_convolution_demo.gif)

可以看到，上图膨胀率为2，每个计算点所覆盖的面积都会向外拓展一圈，将原来的计算点向右向下“挤”，构成如图所示的感受野，卷积核的尺寸为3x3，但感受野的尺寸为5x5，感受野中浅蓝色的格子都是计算点“膨胀”的结果，不参与计算，深蓝色的格子依然按照无膨胀时的规则与卷积核进行计算。类似地，当膨胀率为3时，计算点向外膨胀的像素值为2圈，感受野的大小则变成7x7，但执行计算的计算点数依然是9个。通过在计算点周围进行膨胀，计算点与计算点之间出现了“空洞”，这是膨胀卷积也被称为空洞卷积的原因，很明显，膨胀卷积会改变输出的特征图的尺寸，新的计算公式如下：
$$
H_{o u t}=\frac{H_{\text {in }}+2 P-\operatorname{dilation} *\left(K-1\right)-1}{S}+1
$$
可以看到，膨胀率与核尺寸一样，处于分子中被减掉的部分，因此膨胀率越大，生成的特征图就越小，当步长和膨胀率都小于核尺寸时，增加步长和增加膨胀率都可以降低特征图的尺寸，但如果希望更快地放大感受野，应该选择膨胀卷积，而不是选择增加步长。

膨胀卷积之所以有效，并且没有因为空洞而带来图像信息的损失，是因为：没有膨胀的原始卷积相邻神经元的感受野大概率是重复的；加入膨胀卷积之后，虽然在生成单个神经元时，上一层的特征图上留下了不少未计算的空隙，但深层相邻的神经元却很好地补上了这些没有被计算的部分，使得相邻几个神经元之间很多时候没有重复进行扫描，因此也会带来更大的感受野。同时，将膨胀率调大，并且让膨胀卷积层与普通卷积层串联使用时，单个像素的感受野可以被持续放大，如果再结合池化层进行使用，膨胀卷积放大感受野的性质会更加明显。因为这个非常有用的性质，膨胀卷积在语义分割等任务中有非常优秀的表现，同时膨胀卷积不太适合于小物体的分割，而更适合于大物体的分割。
#### 感受野尺寸
对于第$l$个卷积层/池化层输出的特征图而言，假设核尺寸为正方形、图像尺寸也为正方形，则该图上任意一个神经元的感受野大小为：
$$
r_l=r_{l-1}+\left(k_l-1\right) * \prod_{i=0}^{l-1} s_i
$$
其中，$r_{l-1}$为上一个卷积层/池化层的感受野的大小，$k_l$是$l$层的卷积核/池化核的大小，$s_i$是第$i$层的卷积/池化步长，公式的最后一部分是$l$层之前所有对感受野有影响的卷积、池化层的步长的连乘结果，可见，感受野的大小只与卷积核的大小、各层的步长有关，而与Padding无关，同时，对于第1个卷积层来说，$r_0$和$s_0$都等于1。下面计算LeNet5各层的感受野：

层| 层的参数与结构 | 感受野计算公式         |感受野大小
---|--------|-----------------|---
原始图像|s=1| -               |1
conv1|k=5 s=1| 1+(5-1)*(1)     |5
pool1|k=2 s=2| 5+(2-1)*(1\*1)  |6
conv1|k=5 s=1| 6+(5-1)*(1\*2)  |14
pool2|k=2 s=2| 14+(2-1)*(2\*1) |16

再计算AlexNet各层的感受野：

层| 层的参数与结构 | 感受野计算公式           |感受野大小
---|--------|-------------------|---
原始图像|s=1| -                 |1
conv1|k=11 s=4| 1+(11-1)*(1)      |11
pool1|k=3 s=2| 11+(3-1)*(1\*4)   |19
conv2|k=5 s=1| 19+(5-1)*(4\*2)   |51
pool2|k=3 s=2| 51+(3-1)*(8\*1)   |67
conv3|k=3 s=1| 67+(3-1)*(8\*2)   |99
conv4|k=3 s=1| 99+(3-1)*(16\*1)  |131
conv5|k=3 s=1| 131+(3-1)*(16\*1) |163
pool3|k=3 s=2| 163+(3-1)*(16\*1) |195

除了手动计算，也可以使用自动进行感受野计算的包，网上有一些开源的项目可以用于在PyTorch中计算感受野，例如[https://github.com/Fangyh09/pytorch-receptive-field](https://github.com/Fangyh09/pytorch-receptive-field)、[https://github.com/marcelsan/pytorch-receptive-field](https://github.com/marcelsan/pytorch-receptive-field)和[https://github.com/MLRichter/receptive_field_analysis_toolbox](https://github.com/MLRichter/receptive_field_analysis_toolbox)等，可以将其克隆下来，放入自己的项目中，并直接导入使用即可。下面使用工具实现感受野计算。

In [11]:
# 计算LeNet5的感受野
model = LeNet5()
receptive_field(model, input_size=(1, 32, 32), device='cpu')  # 输入数据结构应该不包含batch_size维度

------------------------------------------------------------------------------
           Layer (type)    map size      start       jump receptive_field 
        0 (Input)          [32, 32]        0.5        1.0               1 
        1 (Conv2d)         [28, 28]        2.5        1.0               5 
        2 (Tanh)           [28, 28]        2.5        1.0               5 
        3 (AvgPool2d)      [14, 14]        3.0        2.0               6 
        4 (Conv2d)         [10, 10]        7.0        2.0              14 
        5 (Tanh)           [10, 10]        7.0        2.0              14 
        6 (AvgPool2d)        [5, 5]        8.0        4.0              16 


OrderedDict([('0',
              OrderedDict([('layer_cls', 'Input'),
                           ('j', 1.0),
                           ('r', 1),
                           ('start', 0.5),
                           ('conv_stage', True),
                           ('output_shape', [-1, 1, 32, 32])])),
             ('1',
              OrderedDict([('layer_cls', 'Conv2d'),
                           ('j', 1.0),
                           ('r', 5),
                           ('start', 2.5),
                           ('input_shape', [-1, 1, 32, 32]),
                           ('output_shape', [-1, 6, 28, 28])])),
             ('2',
              OrderedDict([('layer_cls', 'Tanh'),
                           ('j', 1.0),
                           ('r', 5),
                           ('start', 2.5),
                           ('input_shape', [-1, 6, 28, 28]),
                           ('output_shape', [-1, 6, 28, 28])])),
             ('3',
              OrderedDict([('layer_cls', 'AvgPoo

In [12]:
# 计算AlexNet的感受野
model = AlexNet(dropout=0.2)
receptive_field(model, input_size=(3, 227, 227), device='cpu')

------------------------------------------------------------------------------
           Layer (type)    map size      start       jump receptive_field 
        0 (Input)        [227, 227]        0.5        1.0               1 
        1 (Conv2d)         [55, 55]        5.5        4.0              11 
        2 (MaxPool2d)      [27, 27]        9.5        8.0              19 
        3 (Conv2d)         [27, 27]        9.5        8.0              51 
        4 (MaxPool2d)      [13, 13]       17.5       16.0              67 
        5 (Conv2d)         [13, 13]       17.5       16.0              99 
        6 (Conv2d)         [13, 13]       17.5       16.0             131 
        7 (Conv2d)         [13, 13]       17.5       16.0             163 
        8 (MaxPool2d)        [6, 6]       33.5       32.0             195 


OrderedDict([('0',
              OrderedDict([('layer_cls', 'Input'),
                           ('j', 1.0),
                           ('r', 1),
                           ('start', 0.5),
                           ('conv_stage', True),
                           ('output_shape', [-1, 3, 227, 227])])),
             ('1',
              OrderedDict([('layer_cls', 'Conv2d'),
                           ('j', 4.0),
                           ('r', 11),
                           ('start', 5.5),
                           ('input_shape', [-1, 3, 227, 227]),
                           ('output_shape', [-1, 96, 55, 55])])),
             ('2',
              OrderedDict([('layer_cls', 'MaxPool2d'),
                           ('j', 8.0),
                           ('r', 19),
                           ('start', 9.5),
                           ('input_shape', [-1, 96, 55, 55]),
                           ('output_shape', [-1, 96, 27, 27])])),
             ('3',
              OrderedDict([('layer

可以看到，使用工具计算出来的结果与手动计算的结果一致。需要注意，输出结果中的map_size是特征图尺寸、jump和start是关于感受野中心点的计算参数，最后一列是感受野的大小。对于任意由卷积层和最大池化层构成的架构，都可以使用这个包来进行感受野计算，同时还支持膨胀卷积。
#### 平移不变性
在计算机视觉、尤其是图像识别相关的任务中，不变性（Invariance，Invariant）是至关重要、甚至影响建模和数据处理流程的一种性质，不变性是指，如果能够识别出一张图像中的一个对象，那即便这个对象以完全不同的姿态呈现在别的图像中，依然可以识别出这个对象。对算法而言，不变性意味着在训练集上被成功识别的对象，即便以不同的姿态出现在测试集中，也应该能够被成功识别。人类和卷积网络对图像的“理解”方式不同，导致两者对同样的图像有不同的判断，模型很容易因为学习过度而进入过拟合状态，只认识“自己见过的东西”，泛化水平较低，鲁棒性较低。鲁棒性用于形容一个模型或算法在不同数据、不同环境下的表现是否稳定，一个过拟合的模型的鲁棒性一定是较低的，因为过拟合就意味着不能适用数据的变化。

计算机视觉的使命就是让算法完全实现人类视觉的功能，因此让卷积网络拥有和人眼一样的“不变性”就有巨大的意义：拥有“不变性”不止意味着图像能被算法正确识别，还意味着图像能够被算法正确“理解”，是切实可能提升模型鲁棒性的因素，因此，让CNN具有不变性也是提升测试集效果、减少过拟合、提升鲁棒性的关键步骤。在视觉领域，有很多从不同角度定义的不变性：
- 平移不变性（Translation Invariance）：对图像上的任意一个对象，无论它出现在图像上的什么位置，都能够识别这是同一个对象
- 旋转/视野不变性（Rotation/ViewPoint Inviariance）：对图像上的任意一个对象，无论如何旋转、或更换查看对象的视野，都能识别这是同一个对象
- 尺寸不变性（size Inviarance）
- 明度不变性（Illumination Invariance）
- 镜面不变性（镜面翻转图像），颜色不变性

不同类型的不变性示例如下：
![invariance_of_statue](../assets/invariance_of_statue.png)

在训练深度CNN时，为了增强模型的效果，会尽量让模型去获取更多的“不变性”，大部分深层卷积网络的架构自带一定的“平移不变性”，只要对象的轮廓一致，无论对象出现在图像的哪个位置，卷积网络都能够判断出来，因为对于大部分图像来说，有效信息较为集中的区域的像素值会更大，在卷积操作后得到的特征图上的值也会更大，这是最大池化层能够有效的基础，因此无论有效信息位于特征图的什么位置，在经过最大池化层之后，有效信息都能够被顺利筛选出来，在卷积神经网络中，关键像素被平移后对模型整体准确率的影响相对较小。关于平移不变性的更多细节可参考[https://blog.csdn.net/qq_34107425/article/details/107503099](https://blog.csdn.net/qq_34107425/article/details/107503099)和[https://dengking.github.io/machine-learning/Theory/Deep-learning/Book-deep-learning/Part-II-Deep-Networks-Modern-Practices/9-Convolutional-Networks/CNN-translation-invariance/](https://dengking.github.io/machine-learning/Theory/Deep-learning/Book-deep-learning/Part-II-Deep-Networks-Modern-Practices/9-Convolutional-Networks/CNN-translation-invariance/)，关于卷积神经网络在不同平移程度以及不同深度下的平移不变性的探索可参考[https://divsoni2012.medium.com/translation-invariance-in-convolutional-neural-networks-61d9b6fa03df](https://divsoni2012.medium.com/translation-invariance-in-convolutional-neural-networks-61d9b6fa03df)，其主要结论如下：
- CNN的平移不变性只能够应对“微小的平移”，当物体横向或纵向平移的像素过多时，CNN的平移不变性会衰减
- 卷积+池化层的叠加可以增强CNN的平移不变性（更深的网络拥有更强的平移不变性），同时增强模型的鲁棒性

为了让卷积神经网络具备各类不变性，需要采取更强力的手段，数据增强（Data Augmentation），数据增强是数据科学体系中常用的一种增加数据量的技术，它通过添加略微修改的现有数据、或从现有数据中重新合成新数据来增加数据量。对于图像数据来说，可用的数据增强技术很多，包括旋转、模糊、调节饱和度、放大与缩小、调节亮度、镜面反转、去纹理化、脱色、边缘增强、边缘检测等多种方式。显然，这些变化都是通过改变原始图像的某些像素而实现的，每一种变化都对应着一种可能的“不变性”，对任意一种变化，都会从原始图像生成很多张新的图像放入训练集，这样做可以将训练数据集极速扩大，并且可以让模型快速见到各种“旋转”、“镜像”过的数据，从而让模型获取到各种不变性、能够在测试中识别出更多数据。

但是引入不变性也可能会带来一些问题，例如，在卷积神经网络不断将特征图缩小的过程中，像素之间的“位置”信息是会逐渐损失掉的，当这些信息进入全连接层之后，网络失去了“相对位置”循序，大多数时候平移不变性会提升模型的效果，但对于密集任务（需要对每个像素进行预测的任务）而言，平移不变性可能导致灾难。因此，在为模型添加“不变性”时，应当要清楚需要的是什么，在建立卷积网络之前需要对训练数据有详细的了解，这样才能够将数据处理得更加恰当，无论是AlexNet还是VGG都采用了数据增强来降低模型的过拟合程度，不变性也是现在的研究方向之一。

影响卷积神经网络的效果的模型架构因素还有很多，例如**最后一个卷积层后的特征图的数目**，这个数目也被称为最大感受野上的通道数，有研究表明，通道数越大，CNN的效果会越好。同时对于池化层的作用，有观点认为，真正让信息保持不变、不产生损失的是池化层，同时更深层的卷积网络的平移不变性更强；也有观点认为，池化层不能提供不变性，甚至不能对卷积神经网络的效果有影响，池化层缩小特征图的功能可以由步长等于2的卷积层来替代，关于池化层在卷积神经网络中的作用的讨论可参考[https://zhuanlan.zhihu.com/p/94477174](https://zhuanlan.zhihu.com/p/94477174)。其实，池化层虽然不能提供完美的平移不变性，因为一定会存在信息损失和例外，但从放大感受野的角度而言，池化层对模型的影响是很明显的，对池化层来说，最关键的还是能够快速下采样（down-sampling），即快速减少特征图尺寸、减少模型所需的参数量。
## 3.模型架构对参数量/计算量的影响
在自建架构的时候，除了模型效果之外，还需要关注模型整体的计算效率。深度学习模型天生就需要大量数据进行训练，因此每次训练中的参数量和计算量就格外关键，在设计卷积网络时的目标也是相似预测效果下参数量越少越好。这里的模型参数是需要学习的参数，例如权重weight和常数项bias，任何不需要学习、人为输入的超参数都不在参数量的计算范围内。在卷积神经网络中，有两种方式影响模型的参数量：
1. 这个层自带参数，其参数量与该层的超参数的输入有关
2. 这个层会影响feature map的尺寸，影响整体像素量和计算量，从而影响全连接层的输入

全连接层、BN层通过第1种方式影响参数，而池化、Padding、Stride等操作则通过第2种方法影响参数，卷积层通过2种方式影响参数，Dropout、激活函数等操作不影响参数量。
### 卷积层
#### 参数量计算
一个卷积网络的卷积层究竟包含多少参数量，就是由卷积核的尺寸kernel_size、输入通道数in_channels、输出通道数out_channels共同决定的，公式如下：
$$
N_{\text {parameters }}=\left(K_H * K_W * C_{\text {in }}\right) * C_{\text {out }}+C_{\text {out }}
$$
其中，前面部分是权重weight的数量，后面是偏置bias的数量。下面通过手动计算的方式计算卷积层的参数量。

In [13]:
conv1 = nn.Conv2d(3, 6, 3)  # 3 * 6 * 3 * 3 + 6 = 168
conv1.weight.numel() + conv1.bias.numel()

168

In [14]:
conv2 = nn.Conv2d(6, 16, 5, stride=2, padding=1)  # 6 * 16 * 5 * 5 + 16 = 2416
conv2.weight.numel() + conv2.bias.numel()

2416

可以看到，较大的卷积核、较多的输入和输出都会对参数量影响较大，由于实际中使用的卷积核都很小，所以真正对卷积核参数有影响力的是输出和输入的通道数，在较为复杂的架构中，卷积层的输出数量可能达到256、512、甚至更大，巨大的数字足以让一个卷积层包含的参数达到百万级别，例如VGG16中比较深的几个卷积层的参数都在百万以上。因此，如果希望减小卷积神经网络的参数量，那优先会考虑减少的就是输出的特征图数量，但随着网络加深，特征图是越来越小的，为了学习到更多深入的信息，特征图数量必然会增加，因此，如果希望消减卷积层的参数量，可以使用更少的卷积+池化的组、减少模型深度，如果一定要保持深度，则可以在第1层时就使用较小的特征图数量。
#### 大尺寸卷积核与小尺寸卷积核
在深度卷积网络中，一般都默认使用小卷积核，其中的一个重要原因是：大尺寸卷积核的效果可由多个小尺寸卷积核累积得到。如下图所示：
![conv_big_small_kernel](../assets/conv_big_small_kernel.png)

可以看到，在上面左边是2层核尺寸为3x3的卷积层，第2个卷积层输出的特征图中的1个神经元映射到原始图像上的感受野尺寸为5x5；右边使用了一层5x5的卷积层，也可以得到5x5的感受野。同时，2个3x3卷积层将10x10的特征图缩小为了6x6，一个5x5卷积层也将特征图缩小到了6x6。可以说，在捕获的信息量、压缩尺寸这两个层次上，两个3x3的卷积层和一个5x5的卷积层获得了一样的结果。同理，也可以用3层3x3卷积核的卷积层替代1层7x7的卷积核，更大的卷积核亦然。但是，在参数量方面，1个5x5卷积层在一次扫描中需要的参数是25个，2个3x3卷积层却只需要9 + 9 = 18个，因此2个3x3卷积层所需要的参数更少，当特征图数量巨大时，这一点点参数量的差异会被放大。小卷积核不仅加深了深度，一定程度上让提取出的特征信息更抽象、更复杂，同时也让参数量大幅减少。
#### 1x1卷积核
卷积核到极致就是1x1卷积核，1x1的卷积核上只有1个权重，每次进行卷积操作时，该权重会与原始图像中每个像素相乘，并得到特征图上的新像素，因此1x1卷积也被叫做逐点卷积（Pointwise Convolution），这种计算方式和矩阵*常数一致，同时，其本质与直接给像素直接乘上一个值来改变图像的某些属性的做法很相似。当卷积核尺寸设置为1x1后，参数量为：
$$
N_{\text {parameters }}=C_{\text {in }} * C_{\text {out }}+C_{\text {out }}
$$
可以看到，比起普通卷积核，1x1卷积核的参数量是原来的$\frac{1}{k^2}$（其中k是卷积核尺寸），但由于只有1个像素大小，1x1的卷积核不像普通卷积核一样可以捕捉到特征图/原始图像上一小块的信息，无法识别高和宽维度上相邻元素之间构成的模式。然而，由于1x1卷积核可以在不增加也不减少信息的情况下维持特征图的尺寸，因此1x1卷积核可以完整地将缩小特征图时会损失掉的“位置信息”传输到下一层网络中，这是它在提取特征的一个优势。

1x1卷积的重要作用之一就是**加深CNN的深度**，1x1卷积不会改变特征图的尺寸，因此可以被用于加深CNN的深度，让卷积网络获得更好的特征表达，这个性质是论文《Network in Network》提出的，并在架构NiN中发挥了重要作用，NiN是AlexNet诞生不久之后、同时早于VGG被提出的架构，其各层结构如下：
![nin_structure](../assets/nin_structure.png)

可以看到，在NiN的架构中，存在着一种特殊的层MLP layer，从其结构、操作和输出的特征图来看，MLP layer就是1x1的卷积层。NiN是以每个3x3或5x5卷积层后紧跟2个1x1卷积层组成一个块，并重复3个block达成9层卷积层架构的网络。

1x1卷积核在加深深度方面最关键的作用还是**用在卷积层之间，用于调整输出的通道数，协助大幅度降低计算量和参数量，从而加深网络深度**，这一作用又被称为**跨通道信息交互**。示意图如下：
![kernel_size_1_bottleneck](../assets/kernel_size_1_bottleneck.png)

图中的在核尺寸为1x1的2个卷积层之间包装其他卷积层的架构被称为瓶颈设计（bottleneck design），可简称为瓶颈或bottleneck，它被广泛使用在各种深层网络当中，代表了CNN目前为止最高水平架构之一的残差网络ResNet也使用了瓶颈架构。从直觉上来说，通道数目缩小意味着提取的信息量会变少，但瓶颈设计基本只会出现在超过100层的深度网络中，这样的架构在深度网络中几乎不会造成信息损失，却带来了参数量的骤减，因此瓶颈设计在现实中应用非常广泛。以上图为例，左右两边都输出了256个相同尺寸的特征图，并且所有信息都经过了3x3的卷积核的扫描，但是左边的参数量为590080、右边的参数量为25920，参数量减少超过了20倍，虽然会带来一些性能的下降，但是带来的参数量的降低是很值得的。
#### 减少参数量：分组卷积与深度分离卷积
除了1x1卷积之外，分组卷积（Grouped Convolution）也是一种高效的减少参数量的形式。与线性层中上一层所有的神经元都与下一层所有的神经元相连接类似，普通卷积层的输入的所有通道也与输出层的所有通道连接，根据这个特点与卷积层参数的计算公式可以得到3种消减参数量的办法：消减输入与输出特征图数量；消减每个连接上的核的尺寸；消减输入特征图与输出特征图之间的连接数量。分组卷积就是通过给输入特征图及输出特征图分组来消减连接数量的卷积方式。示意如下：
![conv_group_demo](../assets/conv_group_demo.png)

分组卷积的计算公式如下：
$$
\begin{aligned}
\text { tatal } & =\left(\left(K_H * K_w * \frac{C_{\text {in }}}{g}\right) * \frac{C_{\text {out }}}{g}+\frac{C_{\text {out }}}{g}\right) * g \\
& =\frac{1}{g}\left(K_H * K_w * C_{\text {in }} * C_{\text {out }}\right)+C_{\text {out }}
\end{aligned}
$$
可以看到，分组的存在不影响偏置，偏置只与输出的特征图数量有关。验证如下。

In [15]:
# 普通卷积
conv1 = nn.Conv2d(4, 8, 3)  # 4 * 8 * 3 * 3 + 8 = 296
# 分组卷积
conv2 = nn.Conv2d(4, 8, 3, groups=2)  # 4 * 8 * 3 * 3 / 2 + 8 = 152
conv1.weight.numel() + conv1.bias.numel(), conv2.weight.numel() + conv2.bias.numel()

(296, 152)

可以看到，分组卷积可以有效减少参数量。还有一种特殊的分组卷积，即groups=$C_{in}$的分组卷积称为深度卷积（Depthwise Convolution），其参数量计算公式如下：
$$
\text { parameters }=K_H * K_w * C_{\text {out }}+C_{\text {out }}
$$
可以看到，比起普通卷积，参数量是原来的$\frac{1}{C_{in}}$，当特征图数量巨大时，分组卷积可以节省非常多的参数。图示如下：
![convolution_depthwise_separable](../assets/convolution_depthwise_separable.png)

在上图中，首先进行深度卷积，产出一组特征图，然后再这组特征图的基础上执行1x1卷积、对特征图进行线性变换，这两种卷积组成一个块，称为深度可分离卷积（Depthwise separable convolution），也被称为分离卷积（separable convolution）。对于深度可分离卷积，若不考虑偏置，则整个块的参数量为：
$$
\text { parameters }=K_H * K_w * C_{\text {out }}^{\text {depth }}+C_{\text {in }}^{\text {pair }} * C_{\text {out }}^{\text {pair }}
$$
若普通卷积不考虑偏置，同时1x1卷积层不改变特征图数量，则有$C_{\text {in }}^{\text {pair }}=C_{\text {out }}^{\text {pair }}=C_{\text {out }}^{\text {depth }}$，所以深度可分离卷积的参数与普通卷积的参数的比例为：
$$
\begin{aligned}
\text { ratio } & =\frac{K_H * K_w * C_{\text {out }}^{\text {depth }}+C_{\text {in }}^{\text {pair }} * C_{\text {out }}^{\text {pair }}}{\left(K_H * K_w * C_{\text {in }}^{\text {depth }}\right) * C_{\text {out }}^{\text {pair }}} \\
& =\frac{C_{\text {opt }}^{\text {depth }}}{C_{\text {in }}^{\text {depth }} * C_{\text {out }}^{\text {pair }}}+\frac{C_{\text {in }}}{K_H * K_w * C_{\text {in }}^{\text {depth }}} \\
& =\frac{1}{C_{\text {in }}^{\text {depth }}}+\frac{C_{\text {out }}^{\text {pair }}}{K_H * K_w * C_{\text {in }}^{\text {depth }}}
\end{aligned}
$$
下面进行验证。

In [16]:
# 普通卷积
conv1 = nn.Conv2d(4, 8, 3, bias=False)  # 4 * 8 * 3 * 3 = 288
# 深度可分离卷积
conv2_depthwise = nn.Conv2d(4, 8, 3, groups=4, bias=False)  # 8 * 3 * 3 = 72
conv2_pairwise = nn.Conv2d(8, 8, 1, bias=False)  # 8 * 8 = 64
# 手动与公式计算比例
(conv2_depthwise.weight.numel() + conv2_pairwise.weight.numel()) / conv1.weight.numel(), 1 / 4 + 8 / (
            3 * 3 * 4)  # 两者计算结果一致

(0.4722222222222222, 0.4722222222222222)

深度可分离卷积在2017年的论文《Xception: Deep Learning with Depthwise Separable Convolutions》中被提出，论文中提出，分组卷积核深度可分离卷积不仅可以帮助卷积层减少参数量，而且能够削弱特征图与特征图之间的联系来控制过拟合，是谷歌的深度学习模型GoogLeNet进化版中非常关键的块。
### 全连接层
#### 从卷积层到全连接层
卷积层上减少参数的操作非常丰富，但真正对CNN参数量贡献巨大的是全连接层，数据在进入全连接层时，需要将所有像素拉平，而全连接层中的一个像素点就对应着一个参数，因此全连接层拥有大量参数。卷积神经网络中的全连接层的作用主要有以下2点：
- 作为分类器，实现对数据的分类：卷积层提供了一系列有意义且稳定的特征值，构成了一个与输入图像相比维数更少的特征空间，而全连接层负责学习这个空间上的（可能是非线性的）函数关系，并输出预测结果。
- 作为整合信息的工具，将特征图中的信息进行整合：卷积层生成的特征图是自带位置信息的，任意像素映射到自己的特征图上的位置，与该像素的感受野映射到原图上的位置几乎是一致的，因此为保留特征信息，需要在进行预测之前将所有可能的信息充分混合、进行学习，全连接层能够确保所有信息得到恰当的混合，以保证预测的效果。

全连接层的存在让CNN整体变得更容易过拟合，Dropout、batch normalization等也是为了控制全连接层的过拟合而提出的，使得CNN架构的过拟合不再那么明显。如果模型欠拟合，则可以增加模型的复杂度，对于卷积神经网络中的全连接层来说，增加更多神经元比增加更多层更有效：CNN中的全连接层最多只有3-4层（包括输出层），过多的层会增加计算的负担，还会带来严重的过拟合，同时需要注意，在卷积层和全连接层的连接中，通常全连接的输出神经元个数不会少于输入的通道数，对于全连接层之间的连接，除了是输出层，也很少出现输出神经元少于输入神经元的情况。对全连接层而言，更大的参数代表了更高的复杂度、更强的学习能力、更大的过拟合可能，因此对于小型网络来说，除非数据量庞大或数据异常复杂，尽量不使用过大的参数。

决定全连接层参数数量的有两个因素：连接到卷积层的全连接层的输入神经元个数，以及在全连接层之间设定的输出神经元个数，后者可以根据经验设置，但是前者应该与最后一个卷积层上所有特征图所含的像素量保持一致，要获取到这个值：可以手动一步一步地计算，但是不适用于层数太多的模型；理论上可以用torchinfo等工具，输出每一层的形状，但是这需要有完整的模型；也可以使用另一种构筑神经网络的简单方式，即`nn.Sequential`，它可以将以序列方式从前往后运行的层打包作为一个块，组合成类似于机器学习中的管道（Pipeline）的结构，通过它可以定义复杂的神经网络中的用于实现不同功能的模块。因此可以通过`nn.Sequential`定义前面的所有用于提取图像特征的卷积层、池化层等的模块，并获取到模块的输出及其形状，以此定义后面的全连接层的输入大小。`nn.Sequential`的简单使用如下。

In [17]:
# 定义包含卷积层和池化层的块
features = nn.Sequential(
    nn.Conv2d(3, 6, 3),
    nn.ReLU(inplace=True),
    nn.Conv2d(6, 4, 3),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(2),
    nn.Conv2d(4, 16, 5, stride=2, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(16, 3, 5, stride=3, padding=2),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(2)
)

# 构造数据
images = torch.randint(0, 256, (16, 3, 229, 229), dtype=torch.float)
# 前向传播
features(images).shape  # 获取到了特征提取模块的输出的形状

torch.Size([16, 3, 9, 9])

In [18]:
# 查看各层感受野
receptive_field(features, (3, 229, 229), device='cpu')  # 与前面结果一致

------------------------------------------------------------------------------
           Layer (type)    map size      start       jump receptive_field 
        0 (Input)        [229, 229]        0.5        1.0               1 
        1 (Conv2d)       [227, 227]        1.5        1.0               3 
        2 (ReLU)         [227, 227]        1.5        1.0               3 
        3 (Conv2d)       [225, 225]        2.5        1.0               5 
        4 (ReLU)         [225, 225]        2.5        1.0               5 
        5 (MaxPool2d)    [112, 112]        3.0        2.0               6 
        6 (Conv2d)         [55, 55]        5.0        4.0              14 
        7 (ReLU)           [55, 55]        5.0        4.0              14 
        8 (Conv2d)         [19, 19]        5.0       12.0              30 
        9 (ReLU)           [19, 19]        5.0       12.0              30 
        10 (MaxPool2d)       [9, 9]       11.0       24.0              42 


OrderedDict([('0',
              OrderedDict([('layer_cls', 'Input'),
                           ('j', 1.0),
                           ('r', 1),
                           ('start', 0.5),
                           ('conv_stage', True),
                           ('output_shape', [-1, 3, 229, 229])])),
             ('1',
              OrderedDict([('layer_cls', 'Conv2d'),
                           ('j', 1.0),
                           ('r', 3),
                           ('start', 1.5),
                           ('input_shape', [-1, 3, 229, 229]),
                           ('output_shape', [-1, 6, 227, 227])])),
             ('2',
              OrderedDict([('layer_cls', 'ReLU'),
                           ('j', 1.0),
                           ('r', 3),
                           ('start', 1.5),
                           ('input_shape', [-1, 6, 227, 227]),
                           ('output_shape', [-1, 6, 227, 227])])),
             ('3',
              OrderedDict([('layer_cls

可以看到，使用`nn.Sequential`大大降低了代码量，同时也使得卷积层架构没有在类中逐层定义各个层那么直观。在较为复杂的网络架构中，通常利用`nn.Sequential`来区分网络的不同部分，例如在普通CNN中，卷积层、池化层负责特征提取，全连接层负责整合信息、进行预测，因此可以使用`nn.Sequential`来区别这两部分架构。使用`nn.Sequential`重构VGG16模型网络如下。

In [19]:
class VGG16(nn.Module):
    def __init__(self, num_classes=1000, dropout=0.5):
        super().__init__()
        self.num_classes = num_classes
        self.dropout = dropout

        # 特征提取
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(256, 512, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )

        # 分类
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(512 * 7 * 7, 4096), nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096), nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes), nn.Softmax(dim=-1)
        )

    def forward(self, x):
        out = self.features(x)
        out = torch.flatten(out, 1)
        out = self.classifier(out)
        return out

In [20]:
# 实例化验证模型
model = VGG16(dropout=0.2)
summary(model, (16, 3, 224, 224), device='cpu')

Layer (type:depth-idx)                   Output Shape              Param #
VGG16                                    [16, 1000]                --
├─Sequential: 1-1                        [16, 512, 7, 7]           --
│    └─Conv2d: 2-1                       [16, 64, 224, 224]        1,792
│    └─ReLU: 2-2                         [16, 64, 224, 224]        --
│    └─Conv2d: 2-3                       [16, 64, 224, 224]        36,928
│    └─ReLU: 2-4                         [16, 64, 224, 224]        --
│    └─MaxPool2d: 2-5                    [16, 64, 112, 112]        --
│    └─Conv2d: 2-6                       [16, 128, 112, 112]       73,856
│    └─ReLU: 2-7                         [16, 128, 112, 112]       --
│    └─Conv2d: 2-8                       [16, 128, 112, 112]       147,584
│    └─ReLU: 2-9                         [16, 128, 112, 112]       --
│    └─MaxPool2d: 2-10                   [16, 128, 56, 56]         --
│    └─Conv2d: 2-11                      [16, 256, 56, 56]         29

可以看到，引入了`nn.Sequential`之后，forward方法变得很简单，整体代码量也降低了，在构筑自己的神经网络时，会经常使用`nn.Sequential`来优化卷积层代码结构。
#### 代替全连接层：1x1卷积核与全局平均池化（GAP）
虽然全连接层很有用，但它的参数量带来的计算成本的确是一个很大的问题，因此，找到各种方法用来替代全连接层是一个不错的选择，其中流传比较广泛的方法之一，就是使用1x1卷积核来进行替代全连接层，但实际上这么做的价值其实微乎其微，除了特殊的应用场景之外，很少有实际应用会这么做。对于卷积层来说，只要让特征图的尺寸为1x1，再让卷积核的尺寸也为1x1，就可以实现和普通全连接层一模一样的计算。在计算机视觉中，不包含全连接层，只有卷积层和池化层的卷积网络被叫做全卷积网络（Fully-Convolutional Network，FCN）。1x1卷积核可以在架构上完全替代掉全连接层的例子如下：
![fc_kernel1_in_cnn](../assets/fc_kernel1_in_cnn.png)

可以看到，输出的特征图的个数必须和全连接层上的神经元个数一致，这样才能使用输出的特征图“替代”掉全连接层，但在这样的要求下，经过计算可以得知，卷积层所需要的参数量是更大的，因此，使用1x1卷积层代替全连接层不能减少参数量。1x1卷积核替换全连接层之后带来的最大的好处是解放了输入层对图像尺寸的限制，普通卷积神经网络输入卷积网络的图片的尺寸总是被严格规定的，一旦改变输入尺寸，网络架构就不能再使用，而当整个架构中都只有卷积层时，无论如何调整输入图像的尺寸，网络都可以运行，只是输出层的输出可能形状不同。无论多大的图像都能输出结果的性质在物体检测中有一个有趣的应用，在物体检测中，需要判断一个物体位于图像的什么位置，因此需要使用小于图像尺寸的正方形区域对图像进行“滑窗”识别，在每一个窗口里，我们都需要执行一个单独的卷积网络，用以判断物体是否在这个窗口范围内。

在NiN的架构中，最后一个普通核尺寸的卷积核之后跟着的是MLP层，并且这些MLP层最终将特征图数目缩小到了类别数10，但根据原伦文，实现了全连接层的两个目标（整合信息、输出结果）的实际上是跟在MLP层后的全局平均池化层（Global Average Pooling），作者明确表示，用来替代全连接层的是GAP层，GAP层的本质是池化层，使用了平均池化，其职责就是将上一层传入的特征图（无论多少、大小）都转化成(n_class, 1, 1)结构，为了能够将任何尺寸的特征图化为1x1的尺寸，GAP层所使用的核尺寸就等于输入的特征图尺寸。在NiN网络中，最后一个卷积层的输出是（10, 7, 7），因此全局平均池化层的核尺寸也是7x7，由于只能扫描一次，因此全局平均池化层不设置参数步长和Padding。下面使用普通的平均池化层，并令这个池化层的核尺寸为上层输入的特征图尺寸，来模拟全局平均池化。

In [21]:
# 构造数据
features = torch.rand(16, 10, 7, 7)
# 定义GAP
gap = nn.AvgPool2d(7)
# 前向传播
gap(features).shape

torch.Size([16, 10, 1, 1])

使用1x1卷积核连接GAP的方式，NiN网络中完全没有使用全连接层，这让NiN网络整体的参数量减少不少，同时，GAP作为池化层，没有任何需要学习的参数，这让GAP的抗过拟合能力更强。
#### 复现NiN架构
下面结合NiN的模型架构图，使用`nn.Sequential`复现NiN。
![nin_structure](../assets/nin_structure.png)

In [22]:
class NiN(nn.Module):
    def __init__(self, num_classes=10, dropout=0.5):
        super().__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(3, 192, 5, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(192, 160, 1), nn.ReLU(inplace=True),
            nn.Conv2d(160, 96, 1), nn.ReLU(inplace=True),
            nn.MaxPool2d(3, stride=2),
            nn.Dropout2d(p=dropout)
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(96, 192, 5, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(192, 192, 1), nn.ReLU(inplace=True),
            nn.Conv2d(192, 192, 1), nn.ReLU(inplace=True),
            nn.MaxPool2d(3, stride=2),
            nn.Dropout2d(p=dropout)
        )

        self.block3 = nn.Sequential(
            nn.Conv2d(192, 192, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(192, 192, 1), nn.ReLU(inplace=True),
            nn.Conv2d(192, num_classes, 1, padding=1), nn.ReLU(inplace=True),
            nn.AvgPool2d(7),
            nn.Softmax(dim=-1)
        )

    def forward(self, x):
        out = self.block1(x)
        out = self.block2(out)
        out = self.block3(out)
        return out

In [23]:
# 构造数据
images = torch.rand(16, 3, 32, 32)
# 实例化模型
model = NiN(dropout=0.2)
# 前向传播
model(images).shape

torch.Size([16, 10, 1, 1])

In [24]:
# 查看各层参数
summary(model, (16, 3, 32, 32))

Layer (type:depth-idx)                   Output Shape              Param #
NiN                                      [16, 10, 1, 1]            --
├─Sequential: 1-1                        [16, 96, 15, 15]          --
│    └─Conv2d: 2-1                       [16, 192, 32, 32]         14,592
│    └─ReLU: 2-2                         [16, 192, 32, 32]         --
│    └─Conv2d: 2-3                       [16, 160, 32, 32]         30,880
│    └─ReLU: 2-4                         [16, 160, 32, 32]         --
│    └─Conv2d: 2-5                       [16, 96, 32, 32]          15,456
│    └─ReLU: 2-6                         [16, 96, 32, 32]          --
│    └─MaxPool2d: 2-7                    [16, 96, 15, 15]          --
│    └─Dropout2d: 2-8                    [16, 96, 15, 15]          --
├─Sequential: 1-2                        [16, 192, 7, 7]           --
│    └─Conv2d: 2-9                       [16, 192, 15, 15]         460,992
│    └─ReLU: 2-10                        [16, 192, 15, 15]         -

可以看到，作为9层卷积层、最大特征图数目达到192的网络，NiN的参数量在百万之下，主要还是归功于没有使用全连接层。NiN网络最大的贡献就是让人们意识到了1x1卷积层可能的用途，并且使得舍弃线性层成为了可能，受到NiN网络启发而诞生的GoogLeNet和ResNet都使用了1x1卷积层，并且在各种消减参数的操作下使网络变得更加深。
## 4.前沿卷积神经网络与复现
在深度学习的领域，最前沿、最先进的架构称为state-of-the-art models，简称SOTA，但真正能够进入人们的视野、并被广泛认可为优质架构的架构其实只是凤毛麟角，VGG可以算是准SOTA的架构，也是所有接近或达到SOTA level的架构中唯一一个只使用普通卷积层的架构，其将思路的简洁性发挥到了极致。下面介绍2个可以称为SOTA的卷积神经网络架构。
### GoogLeNet（Inception V1）
#### 动机与思路
VGG非常优秀，但它在ILSVRC 14上却输给了ImageNet数据集上达到6.7%错误率的GoogLeNet，GoogLeNet由谷歌团队与众多大学合作研发，发表于论文《Going deeper with convolutions》，受到NiN网络的启发，谷歌引入了一种全新的网络架构Inception block，并将使用Inception V1的网络架构称为GoogLeNet。相比于VGG，GoogLeNet的效果虽然没有好太多，两个架构在深度上也没有差太多，但Inception展现出比VGG强大许多的潜力：不仅需要的参数量少很多，架构可以达到的上限也更高。随着架构的迭代更新，Inception V3和V4已经是典型的SOTA模型，可以在ImageNet数据集上达到3%的错误率。


自从LeNet5定下了卷积、池化、线性层串联的基本基调，研究者们在相当长的一段时间内都在这条道路上探索，最终抵达的终点就是VGG，VGG找出了能够最大程度加大模型深度、增强模型学习能力的架构，并且利用巧妙的参数设计让特征图的尺寸得以控制，但VGG以及其他串联架构的缺点也是显而易见的，最关键的就是参数过多，各层之间的链接过于稠密（Dense），计算量过大，并且很容易过拟合。为了解决这个问题，提出了多种方法，例如：
- 使用分组卷积、舍弃全连接层等用来消减参数量，让神经元与神经元之间、或特征图与特征图之间的连接数变少，从而让网络整体变得稀疏
- 引入随机的稀疏性，例如使用类似于Dropout的方式来随机地让特征矩阵或权重矩阵中的部分数据为0
- 引入GPU进行计算

但是这些方法存在明显的问题：
- 分组卷积等操作虽然能够有效减少参数量，却也会让架构的学习水平变得不稳定，在神经网络由稠密变得稀疏（Sparse）的过程中，网络的学习能力会波动甚至会下降。
- 随机的稀疏性与GPU计算之间其实是存在巨大矛盾的。现代硬件不擅长处理在随机或非均匀稀疏的数据上的计算，这种不擅长在矩阵计算上表现得尤其明显。

此时就需要在稠密结构和稀疏结构之间进行权衡：稠密结构的学习能力更强，但会因为参数量过于巨大而难以训练；稀疏结构的参数量少，但是学习能力会变得不稳定，并且不能很好地利用现有计算资源。VGG架构选择了传统道路，即在学习能力更强的稠密架构上增加Dropout。但GoogLeNet在设计之初就采用了一种与传统CNN完全不同的构建思路，即使用普通卷积、池化层这些稠密元素组成的块去无限逼近（approximate）一个稀疏架构，从而构造一种参数量与稀疏网络相似的稠密网络。这种思路的核心不是通过减少连接、减少扫描次数等“制造空隙”的方式来降低稠密网络的参数量，而是直接在架构设计上找出一种参数量非常少的稠密网络。在数学中，常常使用稀疏的方式去逼近稠密的结构（这种操作叫稀疏估计sparse approximation），但反过来用稠密结构去近似稀疏架构的情况却几乎没有，因此能否真正实现这种逼近是不得而知的，这种思路可行的一个理论基础在于，稀疏数据可以被聚类成携带高度相似信息的密集数据来加速硬件计算，考虑到神经元和特征图的本质其实都是数据的组合，那稀疏的神经元应该也可以被聚类成携带高度相似信息的密集神经元，如果神经元可以被聚类，那这很可能说明稀疏架构在一定程度上应该可以被稠密架构所替代。基于这样的基本理念，GoogLeNet团队使用了一个复杂的网络架构构造算法，并让算法向着使用稠密成分逼近稀疏架构的方向进行训练，经过大量实验选出了学习能力最强的密集架构及其相关参数，这个架构就是Inception块和GoogLeNet。
#### Inception V1
Inception V1的具体结构如下：
![googlenet_inception_v1](../assets/googlenet_inception_v1.png)

可以看到，Inception块使用了卷积层、池化层并联的方式，一个Inception块中存在4条线路，每条线路称为一个分枝（branch）：
1. 第1条线路上只有一个1x1卷积层，只负责降低通道数；
2. 第2条路线由一个1x1卷积层和一个3x3卷积层组成，本质上是希望使用3x3卷积核进行特征提取，但先使用1x1卷积核降低通道数以此来降低参数量和计算量（降低模型的复杂度）；
3. 第3条线路由一个1x1卷积层和一个5x5卷积层组成，其基本思路与第二条线路一致；
4. 最后一条线路由一个3x3池化层和一个1x1卷积层组成，将池化也当做一种特征提取的方式，并在池化后使用1x1卷积层来降低通道数。

显然，所有的线路都使用了巧妙的参数组合，让特征图的尺寸保持不变，因此在四条线路分别输出结果之后，Inception块将四种方式生成的特征图拼接在一起，形成一组完整的特征图，这组完整的特征图与普通卷积生成的特征图在结构、计算方式上并无区别，因此可以被轻松地输入任意卷积、池化或全连接的结构。Inception块有明显的优势：
- 同时使用多种卷积核可以确保各种类型和层次的信息都被提取出来
- 并联的卷积池化层计算效率更高
- 大量使用1x1卷积层来整合信息，既实现了“聚类信息”又实现了大规模降低参数量，让特征图数量实现了前所未有的增长

GoogLeNet的主体架构如下：
![googlenet_layer_param](../assets/googlenet_layer_param.png)

可以看到，与VGG相比，在相同的输入和输出特征图数量下，GoogLeNet的Inception块的参数量减少到了VGG的普通卷积层的参数量的$\frac{1}{5}$左右，同时Inception块使用的1x1卷积核控制住了整体的参数量，从而解放了特征图的数量，GoogLeNet的特征图数量达到了1024，相比VGG增加了一倍。在整体结构上，Inception内部是稠密部件的并联，而整个GoogLeNet则是数个Inception块与传统卷积结构的串联。GoogLeNet的结构特点如下：
- 在inception的前面有着几个传统的卷积层，并且第一个卷积层采用了和LeNet相似的处理方法：先利用较大的卷积核大幅消减特征图的尺寸，当特征图尺寸下降到28x28后再使用Inception进行处理。
- Inception中虽然已经包含池化层，但Inception之后还是有池化层让特征图尺寸减半。
- 在架构的最后，使用了核尺寸为7x7的、用来替代全连接层的全局平均池化层，这和NiN中的操作一样，在全局平均池化层的最后跟上了一个线性层，用于输出Softmax的分类结果。

除了主体架构之外，GoogLeNet还使用了辅助分类”（auxiliary classifier）以提升模型的性能，这两个分类器的输入分别是inception4a和inception4d的输出结果，结构如下：

层 | 核尺寸/步长| 输入4a输出尺寸| 输入4d输出尺寸
---|---|---|---
平均池化层| 5x5/3 |4x4x512| 4x4x528
卷积层+ReLU |1x1/1 |4x4x128| 4x4x128
全连接层+ReLU| |1024| 1024
Dropout(70%) | | 1024 |1024
全连接层+Softmax | |1000| 1000

这种思路集成了两个浅层网络和一个深层网络的结果来进行学习和判断，也是GoogLeNet的一大创新。将主体架构与辅助分类器结合，我们可以得到GoogLeNet的完整架构，如下：
![googlenet_structure](../assets/googlenet_structure.png)

#### 复现GoogLeNet
相对于前面的经典卷积神经网络，GoogLeNet是一个串联元素中含有更多复杂成分的网络，因此在复现时需要先定义几个单独的元素，之后才能够使用更高效和可重用的方式来复现架构。

首先，因为所有的卷积层后都会接BN层和ReLU激活函数，需要定义新的基础卷积层，来包含激活函数以及BN层的卷积层，实现如下。

In [25]:
class BasicConv2d(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, kernel_size: int, **kwargs):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        return F.relu(out, inplace=True)

In [26]:
# 测试
BasicConv2d(3, 6, 32)

BasicConv2d(
  (conv): Conv2d(3, 6, kernel_size=(32, 32), stride=(1, 1), bias=False)
  (bn): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

接下来定义Inception块，由于Inception块中是并联的结构，存在4个分支，因此不能使用nn.Sequential进行打包，在Inception块中，所有卷积、池化层的输入、输出以及核大小都需要输入，因此需要传入相应的初始化参数。

In [27]:
class Inception(nn.Module):
    def __init__(self, in_channels: int, ch1x1: int, ch3x3red: int, ch3x3: int, ch5x5red: int, ch5x5: int,
                 pool_proj: int):
        super().__init__()
        self.branch1 = BasicConv2d(in_channels, ch1x1, 1)
        self.branch2 = nn.Sequential(
            BasicConv2d(in_channels, ch3x3red, 1),
            BasicConv2d(ch3x3red, ch3x3, 3, padding=1)
        )
        self.branch3 = nn.Sequential(
            BasicConv2d(in_channels, ch5x5red, 1),
            BasicConv2d(ch5x5red, ch5x5, 5, padding=2)
        )
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(3, stride=1, padding=1),
            BasicConv2d(in_channels, pool_proj, 1)
        )

    def forward(self, x):
        out1 = self.branch1(x)
        out2 = self.branch2(x)
        out3 = self.branch3(x)
        out4 = self.branch4(x)
        out = torch.cat([out1, out2, out3, out4], dim=1)  # 拼接4个输出
        return out

In [28]:
# 测试
Inception(192, 64, 96, 128, 16, 32, 32)

Inception(
  (branch1): BasicConv2d(
    (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (branch2): Sequential(
    (0): BasicConv2d(
      (conv): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicConv2d(
      (conv): Conv2d(96, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (branch3): Sequential(
    (0): BasicConv2d(
      (conv): Conv2d(192, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicConv2d(
      (conv): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
      (bn): BatchN

接下来定义辅助分类器（Auxiliary Classifier），其结构与之前的传统卷积网络很相似，可以使用nn.Sequential来打包，并将分类器分为特征提取和分类2部分。

In [29]:
class AuxClassifier(nn.Module):
    def __init__(self, in_channels: int, num_classes: int = 1000):
        super().__init__()
        self.features = nn.Sequential(
            nn.AvgPool2d(5, stride=3),
            BasicConv2d(in_channels, 128, 1)
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 * 4 * 4, 1024), nn.ReLU(inplace=True),
            nn.Dropout(0.7),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        out = self.features(x)
        out = out.reshape(x.shape[0], -1)
        out = self.classifier(out)
        return out

In [30]:
# 测试
AuxClassifier(512)

AuxClassifier(
  (features): Sequential(
    (0): AvgPool2d(kernel_size=5, stride=3, padding=0)
    (1): BasicConv2d(
      (conv): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (classifier): Sequential(
    (0): Linear(in_features=2048, out_features=1024, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.7, inplace=False)
    (3): Linear(in_features=1024, out_features=1000, bias=True)
  )
)

在定义好3个单独的类后，即可依据GoogLeNet的完整架构将完整的模型实现。虽然GoogLeNet的主体结构是串联，但由于存在辅助分类器，因此无法在使用nn.Sequential时单独将辅助分类器的结果提取出来；如果按照辅助分类器存在的为止对架构进行划分，又会导致架构整体在层次上与GoogLeNet的架构图有较大的区别，因此最终还是使用了逐层定义的形式。包含辅助分类器的具体代码如下。

In [31]:
class GoogLeNet(nn.Module):
    def __init__(self, num_classes=1000):
        super().__init__()

        # 块1
        self.conv1 = BasicConv2d(3, 64, 7, stride=2, padding=3)
        self.maxpool1 = nn.MaxPool2d(3, stride=2, padding=1)

        # 块2
        self.conv2 = BasicConv2d(64, 64, 1)
        self.conv3 = BasicConv2d(64, 192, 3, padding=1)
        self.maxpool2 = nn.MaxPool2d(3, stride=2, padding=1)

        # 块3
        self.inception3a = Inception(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = Inception(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(3, stride=2, padding=1)

        # 块4
        self.inception4a = Inception(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = Inception(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = Inception(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = Inception(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = Inception(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(3, stride=2, padding=1)

        # 块5
        self.inception5a = Inception(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = Inception(832, 384, 192, 384, 48, 128, 128)

        # 块6
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.dropout = nn.Dropout2d(0.4)
        self.fc = nn.Linear(1024, num_classes)

        # 辅助分类器
        self.aux1 = AuxClassifier(512, num_classes)
        self.aux2 = AuxClassifier(528, num_classes)

    def forward(self, x):
        out = self.conv1(x)
        out = self.maxpool1(out)

        out = self.conv2(out)
        out = self.conv3(out)
        out = self.maxpool2(out)

        out = self.inception3a(out)
        out = self.inception3b(out)
        out = self.maxpool3(out)

        out = self.inception4a(out)
        out1 = self.aux1(out)  # 辅助分类器1
        out = self.inception4b(out)
        out = self.inception4c(out)
        out = self.inception4d(out)
        out2 = self.aux2(out)  # 辅助分类器2
        out = self.inception4e(out)
        out = self.maxpool4(out)

        out = self.inception5a(out)
        out = self.inception5b(out)

        out = self.avgpool(out)
        out = self.dropout(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)

        return out, out2, out1

In [32]:
# 测试
# 构造数据
images = torch.randint(0, 256, (16, 3, 224, 224), dtype=torch.float)
# 实例化模型
model = GoogLeNet()
# 前向传播
preds, preds2, preds1 = model(images)
preds.shape, preds2.shape, preds1.shape  # 得到了期待的输出

(torch.Size([16, 1000]), torch.Size([16, 1000]), torch.Size([16, 1000]))

In [33]:
# 用torchinfo查看模型的各层结构
summary(model, (16, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
GoogLeNet                                [16, 1000]                --
├─BasicConv2d: 1-1                       [16, 64, 112, 112]        --
│    └─Conv2d: 2-1                       [16, 64, 112, 112]        9,408
│    └─BatchNorm2d: 2-2                  [16, 64, 112, 112]        128
├─MaxPool2d: 1-2                         [16, 64, 56, 56]          --
├─BasicConv2d: 1-3                       [16, 64, 56, 56]          --
│    └─Conv2d: 2-3                       [16, 64, 56, 56]          4,096
│    └─BatchNorm2d: 2-4                  [16, 64, 56, 56]          128
├─BasicConv2d: 1-4                       [16, 192, 56, 56]         --
│    └─Conv2d: 2-5                       [16, 192, 56, 56]         110,592
│    └─BatchNorm2d: 2-6                  [16, 192, 56, 56]         384
├─MaxPool2d: 1-5                         [16, 192, 28, 28]         --
├─Inception: 1-6                         [16, 256, 28, 28]         --
│

可以看到，层次结构中至少有3层，输出了很详细的信息，不便于查看模型结构，此时可以使用depth参数来控制输出结构的层数，如下。

In [34]:
summary(model, (16, 3, 224, 224), depth=1)

Layer (type:depth-idx)                   Output Shape              Param #
GoogLeNet                                [16, 1000]                --
├─BasicConv2d: 1-1                       [16, 64, 112, 112]        9,536
├─MaxPool2d: 1-2                         [16, 64, 56, 56]          --
├─BasicConv2d: 1-3                       [16, 64, 56, 56]          4,224
├─BasicConv2d: 1-4                       [16, 192, 56, 56]         110,976
├─MaxPool2d: 1-5                         [16, 192, 28, 28]         --
├─Inception: 1-6                         [16, 256, 28, 28]         164,064
├─Inception: 1-7                         [16, 480, 28, 28]         389,376
├─MaxPool2d: 1-8                         [16, 480, 14, 14]         --
├─Inception: 1-9                         [16, 512, 14, 14]         376,800
├─AuxClassifier: 1-10                    [16, 1000]                3,188,968
├─Inception: 1-11                        [16, 512, 14, 14]         449,808
├─Inception: 1-12                        [16, 5

此时架构清晰很多，虽然架构复杂，但模型实际只有22层卷积层，加上池化层只有27层，并且GoogLeNet只有1300万参数量，同时其中700万参数都是由分类器上的全连接层带来的，如果不使用辅助分类器，则GoogLeNet的参数会降低一半，而16层的VGG的参数量是1.3亿，两者参数相差超过10倍。从计算量来看，GoogLeNet也全面碾压VGG。在2014年的ILSVRC上，GoogLeNet只以微小的优势打败了VGG19，但其计算效率以及在架构上带来的革新是VGG19无法替代的。

GoogLeNet的架构完美地展现了设计Inception的逻辑：使用密集的成分去接近稀疏的架构，不仅能够像稀疏架构一样参数很少，还能够充分利用密集成分在现代硬件上的计算效率。在2014年之后，谷歌团队数次改进了Inception块和GoogLeNet整体的排布，主要得到了4个版本，在前面的基础上都有所改进，进一步提升了模型效果。目前这一族最强大的是InceptionV3，在经过适当训练之后，Inception V3可以在ImageNet2012年数据集上拿到4.2%的错误率，一度是仅次于深层残差网络（100层以上）的架构。

### ResNet
#### 动机与基本架构
GoogLeNet诞生之后，CNN在ImageNet数据集上能够达到的水平已经非常接近人类识别的表现（约5%），但仍有两个瓶颈一直没有被突破：
1. 网络能够达到的最大深度依然很浅，VGG是19层，GoogLeNet也没有超过25层，CNN还有巨大的潜力
2. 深度网络的训练难度太大，虽然强行堆叠卷积层或inception让网络加深非常容易，但加深后的网络往往收敛困难，损失很高，精度很低

深度网络可以令权重空间上的损失函数图像变得更加平滑，因此经过适当的训练，应该更容易找到好的局部最小值，达成更高的精度。但在实际应用中，深层网络的精度往往更容易达到饱和状态（saturated）：在20层以下时，深度越深网络的学习能力越强，精度越高，但当网络的深度超过20层后，随着深度的增加，网络的精度不仅没有持续升高，反而趋近于平缓，甚至出现下降的趋势。网络深度加深，精度却在降低，这种现象在深度网络的训练中被称为退化（degradation），退化现象的存在导致深度网络在实际任务上的表现常常会不如浅层网络。可见，训练一个很深的神经网络比创造一个很深的神经网络架构难得多。

之所以深度网络在现有训练流程下会退化，一种猜测是，深层网络中的函数关系本质上就比浅层网络中的函数关系更复杂、更难拟合，因此深层网络本质上就比浅层网络更难优化和训练。此时如果希望在不削弱精度的情况下加深网络，可以怎么做呢？假设现在有一个准确率很高的浅层网络，经过训练后它的输出结果是$\sigma$，和真实标签的值非常接近，一种简单粗暴地加深它的方式就是在它的后面添加众多只包含恒等函数（Identity function，输入值和输出值完全一致，表达式为$f(\sigma)=\sigma$）的层，这个函数本身不具备任何的学习能力这样就得到了深层网络，它的输出应该与浅层网络一模一样，它在训练集、测试集上的精度表现应该也与浅层网络一模一样，如此就实现了在不削弱准确率的情况下加深网络的深度。直觉上来说，倘若我们将深层网络的恒等函数更换为任意具备学习能力的层得到新的网络，那新的网络整体的学习能力应该是强于深层网络的，且网络得到的结果应该比$\sigma$更接近真实值，或至少和$\sigma$一样接近真实值，然而试验结果却不是这样，新的网络往往离真实值更远、精度更低，这说明：在现有优化算法、优化思路下，在浅层网络后增加恒等函数很可能就是最优的（optimal）加深网络深度的方式。

基于这样的实验和结论，微软提出了全新的深度网络架构，即残差网络ResNet，其基本思想为：假设增加深度用的最优结构就是恒等函数，利用恒等函数的性质，将用于加深网络深度的结构向更容易拟合和训练的方向设计，从根源上降低深度网络的训练难度。ResNet用来加深网络的结构是残差块（Residual unit、残差单元），残差网络将众多残差单元与普通卷积层串联，以实现在浅层网络后堆叠某种结构、以增加深度的目的。假设输入为$x$、待拟合的真实关系为$H(x)$，用$F(x)$来表示$x$与$H(x)$之间的差异，则有$F(x)=H(x)-x$，此时$F(x)$就是残差Residual。如果恒等函数真的就是最优的增加深度的结构，那在得到适当训练后，残差块本身应该会非常接近于一个恒等函数，因此，输入$x$与$H(x)$之间的客观关系就是$H(x)=x$，残差$F(x)$的取值就会为0。拟合0与x的关系，比拟合一个未知的函数$H(x)$与$x$的关系要容易很多，所以残差是一个比原始函数关系更容易优化的数学对象。如果能够让网络拟合$F(x)$，而不去拟合原始的$H(x)$，那深度网络所面临的拟合难度、优化难度就会大大降低。但是，无论对卷积网络如何调整、让内部如何拟合，卷积网络的输出都必须是特征与标签之间的客观关系，否则建模本身就失去了意义。在这种情况下，可以按照下图的结构进行拟合：
![resnet_residual_unit](../assets/resnet_residual_unit.png)

可以看到，一个残差单元中的输出分为两部分：一部分是输入$x$，我们在不做任何操作的情况下将其输出，另一部分是经过残差单元中的卷积结构输出的拟合结果$F(x)$，而整个残差单元最终的输出是两部分输出的加和，即拟合结果$H(x)=x+F(x)$。在这个结构中，直接从输入到输出的$x$被称为是快捷连接（shortcut connection）或跳跃连接（skip connection），残差单元就是令跳跃连接和普通卷积层并联、并加和其结果的结构，与拟合过程有关、与权重有关的就只有$F(x)$，因此使用这个架构可以强迫卷积网络去拟合$F(x)$。理想状况下，残差块会非常接近恒等函数，所以$H(x)$的值应该非常接近输入$x$本身，残差$F(x)$就会很接近于0，使得卷积网络可以向0的方向拟合。除此之外，残差单元还可以带来以下效果：
1. 残差单元几乎实现了0负担增加深度
2. 残差单元可以大幅加速训练和运算速度
3. 残差网络中不会出现梯度消失

因此，残差单元是设计简单、功能精妙的架构，一个完整的残差网络，就是在普通卷积层和全局平均池化层中间插入数个残差单元的网络，如下图所示，原始论文中提出的残差网络共5种，其深度分别是18层、34层、50层、101层和152层，所有的残差网络都以一个7x7的卷积层开头，后接一个3x3、步长为2的重叠池化层，之后就在保持特征图尺寸一致的情况下重复残差单元，最后再跟上全局平均池化层、线性层和Softmax函数，对于34层及以下的残差网络的每个残差单元中有2个卷积层，更大层的残差网络的每个残差单元中有3个卷积层，同时还使用了瓶颈架构Bottleneck，可以大规模降低参数，让残差网络整体的参数量得到控制。
![resnet_5_versions](../assets/resnet_5_versions.png)
#### 技术细节
从架构上来看，残差网络似乎并不是太难，但是其中包含了很多技术细节，在复现时需要格外注意。

（1）Padding：和其他模型一样，残差网络的架构中也隐藏了许多信息，例如每个卷积层后的ReLU激活函数、Batch Normalization、每一层的padding和stride，只有1x1和3x3两种卷积核，为了保持特征图的尺寸不变，3x3卷积核搭配的padding设为1。

（2）stride：随着残差网络的加深，特征图的尺寸也是在逐渐变小的，残差网络的特征图总共缩小了5次，每次长宽都折半，最终尺寸为7x7，因此网络的整体架构也被分成了5个部分，即5层，每层重复的残差单元是一致的，每层中包含的残差单元或瓶颈结构是块，因为在残差单元或瓶颈结构中都不存在池化层，说明对应的降维都是由步长为2的卷积层来完成的，这些步长为2的卷积层是本层中第一个残差单元或瓶颈结构的第一个卷积层。

（3）跳跃连接上的卷积层：$x$和$F(x)$都是一系列的特征图，两者相加则意味着对应特征图上对应位置的像素值一一加和，因为原始特征图是直接通过跳跃连接进行输出的，所以尺寸不会发生变化，而经过瓶颈架构或残差单元，卷积层缩小了特征图尺寸，其输出的特征图$F(x)$就无法与原始特征图$x$相加了。因此，每当卷积层缩小特征图尺寸时，也需要在跳跃连接上加入核为1x1、步长为2的卷积层用于缩小原始特征图的尺寸。同样需要注意的还有特征图数量的变化，这是整个架构中最让人头疼的部分。同时，如果特征图数量不同，矩阵也无法相加，因此无论特征图尺寸是否发生变化，都需要确保跳跃连接上1x1卷积层的输出数量与该block内部最后一个卷积层的输出数量一致。对于残差单元而言，一个块内部的特征图数量都是一致的，因此跳跃连接的特征图数量不总是需要进行转换；但对于瓶颈架构而言，前两个卷积层共享输出的特征图数量middle_out，最后一个卷积层输出的特征图数量是middle_out的四倍，无论特征图的尺寸是否发生变化，跳跃连接上都一定要有1x1卷积层来确保被整理为$x$与$F(x)$相同的结构。图示如下：
![resnet_conv_in_x](../assets/resnet_conv_in_x.png)

（4）BN层与ReLU激活函数的位置：对于不缩减特征图尺寸的残差单元来说，卷积核拟合出的$F(x)$需要BN来调整数据分布；对于跳跃连接来说，通过跳跃连接被直接传到输出口的原始数据$x$不需要BN，带有1x1卷积层的跳跃连接也需要BN。原则上来说，ReLU会出现在每一个BN层的后面，但对于残差单元或瓶颈架构来说，整体输出应当是$H(x)$，因此ReLU函数需要被放置在$x$与$F(x)$相加之后。

（5）参数初始化的位置：在残差网络中，初始化参数的目的是令残差单元或瓶颈架构尽量与恒等函数相似，如果希望整个块与恒等函数尽量相似，就需要尽量让$F(x)$的值为0，对于残差块和平镜架构而言，最后一个能够作用于$F(x)$的值的架构是最后一个卷积层后的BN层。Batch Normlization归一化实现的操作为$\text { output }=\frac{x-E[x]}{\sqrt{\operatorname{Var}[x]+\epsilon}} * \gamma+\beta$，其中$\beta$和$\gamma$是需要学习的参数，论文《Accurate, Large Minibatch SGD: Training ImageNet in 1 Hour》指出，如果将$\beta$和$\gamma$都设置为0，那任何经过BN层的数据都会为0，同时因为nn.BatchNorm2d中默认$\beta$为0，所以只需要将$\gamma$参数设置为0即可。

（6）特征图数量频繁、多样地变化：残差网络比之前的任何网络都要深得多，需要使用具有一定通用性的代码来实现不同的层，不能按照每层的输入、输出的方式来控制卷积的输入和输出，而不同的卷积层需要共享输入的超参数，同时，超参数的数量不能很多。在特征图频繁、多样变化的情况下，必须要区别不同场景下的特征图数量的变化规律。对于50层以下的残差网络来说，增加深度的结构是残差单元，因此特征图数量会存在以下2种情况：
1. 保持不变：conv1与conv2_x连接，与每个层内部不同的残差单元连接，特征图数量都是不变的
2. 翻倍：在不同的层之间连接时，下一个层输出的特征图数量是上一个layers输出数量的2倍。

对于50层以上的残差网络来说，深层残差网络中用来增加深度的结构是带有跳跃连接的瓶颈架构，一个瓶颈架构中涉及到3个卷积层，特征图数量会存在如下4种情况：
1. 保持不变:conv1与conv2_x连接，与一个瓶颈架构内嵌两个卷积层连接，特征图数量保持不变。
2. 扩大上层输出数量的4倍：在一个瓶颈架构内，前两个卷积层共享特征图数目（用middle_out进行表示），第3个1x1卷积层的输出特征图数量是4 * middle_out。
3. 缩小为上层输出数量的$\frac{1}{4}$：在一个层内，上一个瓶颈架构与下一个瓶颈架构连接时，特征图数目需要从4 * middle_out恢复成middle_out。
4. 缩小为上层输出数量的$\frac{1}{2}$：在不同的层之间连接时，上一个瓶颈架构输出的特征图数量是下一个瓶颈架构输出特征图数量的2倍。

在具体实现时，必须要区别上述所有不同场景中不同的特征图数量变化。同时，希望不同层的残差网络可以共享一个类，在一个类的基础上，使用不同的参数来控制内部结构及所有卷积层上的变化，并且代码要尽量简洁和清晰。
#### 逐步复现残差网络
和复现GoogLeNet时一样，先从简单的、基本的元素开始定义，卷积层主要有两种，3x3卷积层与1x1卷积层，每个卷积层后面都需要跟上BN层，BN层上需要完成参数初始化。

In [35]:
class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, is_block_last=False):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride,
                              padding=1 if kernel_size == 3 else 0, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        if is_block_last:  # 块中的最后一层，BN层进行0初始化
            nn.init.constant_(self.bn.weight, 0)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        return out

In [36]:
# 测试3x3卷积网络
BasicConv2d(3, 6, 3)

BasicConv2d(
  (conv): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [37]:
# 测试1x1卷积网络
BasicConv2d(3, 6, 1, is_block_last=True).bn.weight

Parameter containing:
tensor([0., 0., 0., 0., 0., 0.], requires_grad=True)

接下来定义残差单元，一个残差单元中只包含两个卷积层和一个跳跃连接。实现如下。

In [38]:
class ResidualUnit(nn.Module):
    def __init__(
            self,
            layer_channels: int,  # 当前层的通道数
            block_first_stride: int = 1,  # 当前残差单元的第1个卷积层的步长
            block_in_channels=None
    ):
        super().__init__()

        if block_first_stride != 1:  # 特征图需要缩小时，当前块的输入特征图数量=输出特征图数量的一半
            block_in_channels = layer_channels // 2
        else:  # 特征图不变时，当前块的输入特征图数量=输出特征图数量
            block_in_channels = layer_channels

        # 拟合部分：输出F(x)
        self.residual = nn.Sequential(
            BasicConv2d(block_in_channels, layer_channels, 3, stride=block_first_stride),
            nn.ReLU(inplace=True),
            BasicConv2d(layer_channels, layer_channels, 3, is_block_last=True)

        )

        # 跳跃连接，输出x
        self.shortcut = None
        if block_first_stride != 1:  # 缩小特征图时，也需要跳跃连接加卷积层，以保证可以相加
            self.shortcut = BasicConv2d(block_in_channels, layer_channels, 1, stride=block_first_stride)

        # 加和之后的激活函数
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        # 拟合结果
        fx = self.residual(x)
        # 跳跃连接
        if self.shortcut:
            identity = self.shortcut(x)
        else:
            identity = x

        out = self.relu(fx + identity)
        return out

In [39]:
# 测试特征图不变
feature_map = torch.rand(16, 64, 56, 56)
resnet18_conv2_0 = ResidualUnit(64)
resnet18_conv2_0(feature_map).shape

torch.Size([16, 64, 56, 56])

In [40]:
# 测试特征图缩小
resnet18_conv3_0 = ResidualUnit(128, block_first_stride=2)
resnet18_conv3_0(feature_map).shape

torch.Size([16, 128, 28, 28])

再实现瓶颈结构。

In [41]:
class Bottleneck(nn.Module):
    def __init__(
            self,
            layer_channels: int,  # 当前层的通道数，即middle_out
            block_first_stride: int = 1,  # 当前块的第1个卷积层的步长
            block_in_channels: int = None  # 当前块的输入通道数，如果当前块是conv2的第1个瓶颈结构时为固定值64，需要特别指定
    ):
        super().__init__()

        # 输出通道数
        block_out_channels = layer_channels * 4
        # 输入通道数：当前块不是位于conv1之后
        if not block_in_channels:
            if block_first_stride != 1:  # 缩小特征图，两个层连接、层中的第1个瓶颈结构，通道数减小一半
                block_in_channels = layer_channels * 2
            else:  # 不缩小特征图，同一层内部的非第1个瓶颈结构
                block_in_channels = layer_channels * 4

        # 拟合部分：输出F(x)
        self.residual = nn.Sequential(
            BasicConv2d(block_in_channels, layer_channels, 1, stride=block_first_stride),
            nn.ReLU(inplace=True),
            BasicConv2d(layer_channels, layer_channels, 3),
            nn.ReLU(inplace=True),
            BasicConv2d(layer_channels, block_out_channels, 1, is_block_last=True)
        )
        # 跳跃连接：输出x
        self.shortcut = BasicConv2d(block_in_channels, block_out_channels, 1, stride=block_first_stride)

        # 加和之后的激活函数
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        # 拟合结果
        fx = self.residual(x)
        # 跳跃连接
        x = self.shortcut(x)
        # 加和
        out = self.relu(fx + x)
        return out

In [42]:
# 测试：conv1后的第1个瓶颈结构
feature_map = torch.rand(16, 64, 56, 56)
resnet101_conv2_0 = Bottleneck(64, block_in_channels=64)
resnet101_conv2_0(feature_map).shape  # 输出特征图数量扩大4倍，特征图尺寸不变

torch.Size([16, 256, 56, 56])

In [43]:
# 测试：缩小特征图尺寸
feature_map = torch.rand(16, 256, 56, 56)
resnet101_conv3_0 = Bottleneck(128, block_first_stride=2)
resnet101_conv3_0(feature_map).shape  # 特征图数量扩大2倍，特征图尺寸缩小一半

torch.Size([16, 512, 28, 28])

In [44]:
# 测试：不需要缩小特征图尺寸
feature_map = torch.rand(16, 512, 28, 28)
resnet101_conv3_1 = Bottleneck(128)
resnet101_conv3_1(feature_map).shape  # 输出特征图数量和特征图尺寸不变

torch.Size([16, 512, 28, 28])

现在，需要定义一个函数，用来生成每个层中所有的块。

In [45]:
def make_mayers(
        block: Type[Union[ResidualUnit, Bottleneck]],  # 设置选择残差单元或者瓶颈结构
        layer_channels: int,  # 当前层的输入通道数
        num_blocks: int,  # 当前层的块的数量
        is_after_conv1: bool = False  # 是否位于conv1之后

):
    layers = [
        (
            block(layer_channels, block_in_channels=64) if is_after_conv1
            else block(layer_channels, block_first_stride=2)
        ) if idx == 0 else block(layer_channels)
        for idx in range(num_blocks)
    ]

    return nn.Sequential(*layers)

In [46]:
# 测试：残差单元，conv1之后
resnet34_conv2 = make_mayers(ResidualUnit, 64, 3, True)
summary(resnet34_conv2, (16, 64, 56, 56), depth=1)

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [16, 64, 56, 56]          --
├─ResidualUnit: 1-1                      [16, 64, 56, 56]          73,984
├─ResidualUnit: 1-2                      [16, 64, 56, 56]          73,984
├─ResidualUnit: 1-3                      [16, 64, 56, 56]          73,984
Total params: 221,952
Trainable params: 221,952
Non-trainable params: 0
Total mult-adds (G): 11.10
Input size (MB): 12.85
Forward/backward pass size (MB): 308.28
Params size (MB): 0.89
Estimated Total Size (MB): 322.01

In [47]:
# 测试：瓶颈结构，conv1之后
resnet101_conv2 = make_mayers(Bottleneck, 64, 3, True)
summary(resnet101_conv2, (16, 64, 56, 56), depth=2)

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [16, 256, 56, 56]         --
├─Bottleneck: 1-1                        [16, 256, 56, 56]         --
│    └─Sequential: 2-1                   [16, 256, 56, 56]         58,112
│    └─BasicConv2d: 2-2                  [16, 256, 56, 56]         16,896
│    └─ReLU: 2-3                         [16, 256, 56, 56]         --
├─Bottleneck: 1-2                        [16, 256, 56, 56]         --
│    └─Sequential: 2-4                   [16, 256, 56, 56]         70,400
│    └─BasicConv2d: 2-5                  [16, 256, 56, 56]         66,048
│    └─ReLU: 2-6                         [16, 256, 56, 56]         --
├─Bottleneck: 1-3                        [16, 256, 56, 56]         --
│    └─Sequential: 2-7                   [16, 256, 56, 56]         70,400
│    └─BasicConv2d: 2-8                  [16, 256, 56, 56]         66,048
│    └─ReLU: 2-9                         [16, 256, 56, 56]   

In [48]:
# 测试：瓶颈结构，后边的层
resnet101_conv4 = make_mayers(Bottleneck, 256, 23)
summary(resnet101_conv4, (16, 512, 28, 28), depth=1)

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [16, 1024, 14, 14]        --
├─Bottleneck: 1-1                        [16, 1024, 14, 14]        1,512,448
├─Bottleneck: 1-2                        [16, 1024, 14, 14]        2,167,808
├─Bottleneck: 1-3                        [16, 1024, 14, 14]        2,167,808
├─Bottleneck: 1-4                        [16, 1024, 14, 14]        2,167,808
├─Bottleneck: 1-5                        [16, 1024, 14, 14]        2,167,808
├─Bottleneck: 1-6                        [16, 1024, 14, 14]        2,167,808
├─Bottleneck: 1-7                        [16, 1024, 14, 14]        2,167,808
├─Bottleneck: 1-8                        [16, 1024, 14, 14]        2,167,808
├─Bottleneck: 1-9                        [16, 1024, 14, 14]        2,167,808
├─Bottleneck: 1-10                       [16, 1024, 14, 14]        2,167,808
├─Bottleneck: 1-11                       [16, 1024, 14, 14]        2,167,808
├─Bottle

现在实现完整的残差网络。

In [49]:
class ResNet(nn.Module):
    def __init__(
            self,
            block: Type[Union[ResidualUnit, Bottleneck]],  # 指定残差单元或瓶颈结构
            layer_num_blocks: List[int],  # 各层的块的数量
            num_classes: int = 1000,  # 类别数
    ):
        super().__init__()

        # 普通卷积层+池化层
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, 7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, stride=2, padding=1)
        )

        # 残差块或瓶颈结构
        self.layer2 = self._make_mayers(block, 64, layer_num_blocks[0], True)
        self.layer3 = self._make_mayers(block, 128, layer_num_blocks[1])
        self.layer4 = self._make_mayers(block, 256, layer_num_blocks[2])
        self.layer5 = self._make_mayers(block, 512, layer_num_blocks[3])

        # 全局平均池化
        self.avgpool = nn.AdaptiveAvgPool2d(1)

        # 分类器
        if block == ResidualUnit:
            self.classifier = nn.Linear(512, num_classes)
        else:
            self.classifier = nn.Linear(2048, num_classes)

    def _make_mayers(
            self,
            block: Type[Union[ResidualUnit, Bottleneck]],  # 设置选择残差单元或者瓶颈结构
            layer_channels: int,  # 当前层的输入通道数
            num_blocks: int,  # 当前层的块的数量
            is_after_conv1: bool = False  # 是否位于conv1之后

    ):
        layers = [
            (
                block(layer_channels, block_in_channels=64) if is_after_conv1
                else block(layer_channels, block_first_stride=2)
            ) if idx == 0 else block(layer_channels)
            for idx in range(num_blocks)
        ]

        return nn.Sequential(*layers)

    def forward(self, x):
        # 普通卷积层
        out = self.layer1(x)
        # 残差单元或瓶颈结构
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)

        # 汇总特征
        out = self.avgpool(out)  # (B, C, 1, 1)
        # 拉平特征图
        out = out.squeeze()
        # 分类
        out = self.classifier(out)

        return out

In [50]:
# 测试浅层网络
resnet34 = ResNet(ResidualUnit, [3, 4, 6, 3])
summary(resnet34, (16, 3, 224, 224), depth=2)

Layer (type:depth-idx)                        Output Shape              Param #
ResNet                                        [16, 1000]                --
├─Sequential: 1-1                             [16, 64, 56, 56]          --
│    └─Conv2d: 2-1                            [16, 64, 112, 112]        9,408
│    └─BatchNorm2d: 2-2                       [16, 64, 112, 112]        128
│    └─ReLU: 2-3                              [16, 64, 112, 112]        --
│    └─MaxPool2d: 2-4                         [16, 64, 56, 56]          --
├─Sequential: 1-2                             [16, 64, 56, 56]          --
│    └─ResidualUnit: 2-5                      [16, 64, 56, 56]          73,984
│    └─ResidualUnit: 2-6                      [16, 64, 56, 56]          73,984
│    └─ResidualUnit: 2-7                      [16, 64, 56, 56]          73,984
├─Sequential: 1-3                             [16, 128, 28, 28]         --
│    └─ResidualUnit: 2-8                      [16, 128, 28, 28]         230,144

In [51]:
# 测试深层网络
resnet152 = ResNet(Bottleneck, [3, 8, 36, 3])
summary(resnet152, (16, 3, 224, 224), depth=2, device='cpu')

Layer (type:depth-idx)                        Output Shape              Param #
ResNet                                        [16, 1000]                --
├─Sequential: 1-1                             [16, 64, 56, 56]          --
│    └─Conv2d: 2-1                            [16, 64, 112, 112]        9,408
│    └─BatchNorm2d: 2-2                       [16, 64, 112, 112]        128
│    └─ReLU: 2-3                              [16, 64, 112, 112]        --
│    └─MaxPool2d: 2-4                         [16, 64, 56, 56]          --
├─Sequential: 1-2                             [16, 256, 56, 56]         --
│    └─Bottleneck: 2-5                        [16, 256, 56, 56]         75,008
│    └─Bottleneck: 2-6                        [16, 256, 56, 56]         136,448
│    └─Bottleneck: 2-7                        [16, 256, 56, 56]         136,448
├─Sequential: 1-3                             [16, 512, 28, 28]         --
│    └─Bottleneck: 2-8                        [16, 512, 28, 28]         379,3

可以看到，ResNet的参数量与计算量都很小，34层的残差网络的参数量为2200万，152层的残差网络参数量也才刚过亿，但却可以达到的深度非常深，相比VGG有着很大的优势。从模型效果来看，残差网络毫无疑问是现有的最顶尖的模型之一，几乎所有大型数据集的得分榜单前几名都是残差网络占据。除了基本网络，残差网络还有许多有效、强大的变体，如ResNeXt（在残差网络上加入了并联结构）、WideResNet（增加了模型的宽度，目前为止最强大的模型）等，更多内容可参考[https://towardsdatascience.com/illustrated-10-cnn-architectures-95d78ace614d](https://towardsdatascience.com/illustrated-10-cnn-architectures-95d78ace614d)。